In [ ]:
## GOAL: Prediction of setter behavior

In [ ]:
import base64
import importlib
import os

import dvwtools.read as dv
import dvwtools.stats as dvstats
import numpy as np
import pandas as pd
import sklearn.metrics
from IPython.core.display import display
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

import helper_classification as helpclass

importlib.reload(dvstats)
importlib.reload(dv)
importlib.reload(helpclass)

pd.set_option("max_columns", None)
pd.set_option("mode.chained_assignment", "raise")

In [ ]:
# Loads the image of the volleyball court
path_image = os.path.join("assets", "mezzoCampo_up.png")
path_image_set = os.path.join("assets", "mezzoCampo.png")

# Just some handling of the volleyball court image
encoded_image_up = base64.b64encode(open(path_image, "rb").read())
encoded_image_down = base64.b64encode(open(path_image_set, "rb").read())

In [ ]:
TUNE = False

<h1>Predicting setter distribution in high-level volleyball teams<span class="tocSkip"></span></h1>


December 2021

Andrea Biasioli

Aknowledgments: I would like to thank César Hernández González (Head Coach of Korea NT and Assistant Coach of Vakifbank Istanbul) for the data used in this work.


# Main objective
Notion of side-out: in a team, the setter is responsible for the second ball touch (set), occurring after the first touch (reception), delivering the ball to one of the attackers for the third and last allowed touch (attack). See image below.

While the choice might seem (and sometimes is) random, most of the time there is a reasoning behind the setter's choice. It might be related to the height of the different blockers, or the presence of a strong attacker in the lineup, other factors, or none of the above.

**The goal of the model is to *predict* the setter choice in side-out**, with the available information. More specifically, this study targets the Italian club team **Imoco Conegliano** and its setter **Asia Wolosz**; Conegliano currently holds the world record for most consecutive wins (76) and competing in the 2021 Volleyball Club World Championship.

The importance of the analysis is related to **limiting the opponent team side-out efficiency by learning in advance what are the patterns that are more likely to repeat** during the game.

The implications of this study are:
- **if a simple and explainable model, like a decision tree, can be developed, then the decision tree itself could be printed out for players to aid their decisions** in blocking. A reasonable goal for the model would be to help limit the competence area that a middle blocker should be able to cover. The assumption is that if a middle blocker competence area is only 4.5m long (instead of the full 9m of the net) she might have a better chance of success in controlling the opponent attack.
- **if a complex model is needed, then it could be implemented in a live system using the laptop on the bench for real-time predictions**. The model would predict the setter choice right before the side-out occurs, using the  information available beforehand.
- the study of patterns is often done manually, by analyzing hours and hours of video, and it mostly follows a pattern of its own. For example, divide the rallies by setter rotation, subdivide by setter calls, and look at occurrence frequencies. While this analysis is largely successful and unbiased, it can fail to capture larger dynamics (in the example above, dynamics that extend to multiple rotations or calls) because of the high variance of the highly specific video analysis. This study can help by providing both a sanity-check benchmark for the video analysis (**verify that the results from automated and manual system indeed match**), and could also **highlight what the most important factors are without any prior knowledge of the setter**. For example, the most important factor for one setter might not be the specific setter call, but the height of the different blockers instead, while for another setter it could be uniquely the location of the reception.


<img alt="sideout" src="images/cumulated.jpg" width="850"/>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Predicting-set-distribution-in-volleyball" data-toc-modified-id="Predicting-set-distribution-in-volleyball-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Predicting set distribution in volleyball</a></span></li><li><span><a href="#Main-objective" data-toc-modified-id="Main-objective-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Main objective</a></span></li><li><span><a href="#Introduction-to-the-dataset" data-toc-modified-id="Introduction-to-the-dataset-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Introduction to the dataset</a></span></li><li><span><a href="#Data-cleaning" data-toc-modified-id="Data-cleaning-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Data cleaning</a></span><ul class="toc-item"><li><span><a href="#Descriptive-statistics" data-toc-modified-id="Descriptive-statistics-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Descriptive statistics</a></span></li><li><span><a href="#set_x" data-toc-modified-id="set_x-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>set_x</a></span></li><li><span><a href="#set_y" data-toc-modified-id="set_y-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>set_y</a></span></li><li><span><a href="#reception_x" data-toc-modified-id="reception_x-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>reception_x</a></span></li><li><span><a href="#reception_y" data-toc-modified-id="reception_y-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>reception_y</a></span></li><li><span><a href="#attack_x" data-toc-modified-id="attack_x-4.6"><span class="toc-item-num">4.6&nbsp;&nbsp;</span>attack_x</a></span></li><li><span><a href="#serve_x" data-toc-modified-id="serve_x-4.7"><span class="toc-item-num">4.7&nbsp;&nbsp;</span>serve_x</a></span></li><li><span><a href="#ScoreScaled" data-toc-modified-id="ScoreScaled-4.8"><span class="toc-item-num">4.8&nbsp;&nbsp;</span>ScoreScaled</a></span></li><li><span><a href="#ScoreDelta" data-toc-modified-id="ScoreDelta-4.9"><span class="toc-item-num">4.9&nbsp;&nbsp;</span>ScoreDelta</a></span></li><li><span><a href="#sideout_no" data-toc-modified-id="sideout_no-4.10"><span class="toc-item-num">4.10&nbsp;&nbsp;</span>sideout_no</a></span></li><li><span><a href="#block-heights" data-toc-modified-id="block-heights-4.11"><span class="toc-item-num">4.11&nbsp;&nbsp;</span>block heights</a></span></li><li><span><a href="#call" data-toc-modified-id="call-4.12"><span class="toc-item-num">4.12&nbsp;&nbsp;</span>call</a></span></li><li><span><a href="#players-codes" data-toc-modified-id="players-codes-4.13"><span class="toc-item-num">4.13&nbsp;&nbsp;</span>players codes</a></span></li><li><span><a href="#IsSetterBlocking" data-toc-modified-id="IsSetterBlocking-4.14"><span class="toc-item-num">4.14&nbsp;&nbsp;</span>IsSetterBlocking</a></span></li></ul></li><li><span><a href="#Data-preparation-and-feature-engineering" data-toc-modified-id="Data-preparation-and-feature-engineering-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Data preparation and feature engineering</a></span><ul class="toc-item"><li><span><a href="#serve_x" data-toc-modified-id="serve_x-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>serve_x</a></span></li><li><span><a href="#sideout_no" data-toc-modified-id="sideout_no-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>sideout_no</a></span></li><li><span><a href="#ScoreScaled" data-toc-modified-id="ScoreScaled-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>ScoreScaled</a></span></li><li><span><a href="#call" data-toc-modified-id="call-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>call</a></span></li><li><span><a href="#players-codes-(MB1Code,-MB2Code,-OH1Code,-OH2Code)" data-toc-modified-id="players-codes-(MB1Code,-MB2Code,-OH1Code,-OH2Code)-5.5"><span class="toc-item-num">5.5&nbsp;&nbsp;</span>players codes (MB1Code, MB2Code, OH1Code, OH2Code)</a></span></li><li><span><a href="#set_y" data-toc-modified-id="set_y-5.6"><span class="toc-item-num">5.6&nbsp;&nbsp;</span>set_y</a></span></li><li><span><a href="#ScoreDelta" data-toc-modified-id="ScoreDelta-5.7"><span class="toc-item-num">5.7&nbsp;&nbsp;</span>ScoreDelta</a></span></li></ul></li><li><span><a href="#Target-variables" data-toc-modified-id="Target-variables-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Target variables</a></span><ul class="toc-item"><li><span><a href="#attacker" data-toc-modified-id="attacker-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>attacker</a></span></li><li><span><a href="#region_coordinate" data-toc-modified-id="region_coordinate-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>region_coordinate</a></span></li><li><span><a href="#binarized_attacker" data-toc-modified-id="binarized_attacker-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>binarized_attacker</a></span></li><li><span><a href="#Final-choice" data-toc-modified-id="Final-choice-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>Final choice</a></span></li></ul></li><li><span><a href="#Target:-region_combination" data-toc-modified-id="Target:-region_combination-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Target: region_combination</a></span><ul class="toc-item"><li><span><a href="#Label-encoding" data-toc-modified-id="Label-encoding-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Label encoding</a></span></li><li><span><a href="#Preprocessing-pipeline" data-toc-modified-id="Preprocessing-pipeline-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Preprocessing pipeline</a></span></li><li><span><a href="#Correlation-analysis" data-toc-modified-id="Correlation-analysis-7.3"><span class="toc-item-num">7.3&nbsp;&nbsp;</span>Correlation analysis</a></span></li><li><span><a href="#Model-baseline" data-toc-modified-id="Model-baseline-7.4"><span class="toc-item-num">7.4&nbsp;&nbsp;</span>Model baseline</a></span></li><li><span><a href="#No-tuning-results" data-toc-modified-id="No-tuning-results-7.5"><span class="toc-item-num">7.5&nbsp;&nbsp;</span>No-tuning results</a></span></li><li><span><a href="#Most-important-features" data-toc-modified-id="Most-important-features-7.6"><span class="toc-item-num">7.6&nbsp;&nbsp;</span>Most important features</a></span></li><li><span><a href="#Reduced-set-of-features" data-toc-modified-id="Reduced-set-of-features-7.7"><span class="toc-item-num">7.7&nbsp;&nbsp;</span>Reduced set of features</a></span></li><li><span><a href="#Random-forest-tuning" data-toc-modified-id="Random-forest-tuning-7.8"><span class="toc-item-num">7.8&nbsp;&nbsp;</span>Random forest tuning</a></span><ul class="toc-item"><li><span><a href="#Effect-of-number-of-trees" data-toc-modified-id="Effect-of-number-of-trees-7.8.1"><span class="toc-item-num">7.8.1&nbsp;&nbsp;</span>Effect of number of trees</a></span></li><li><span><a href="#Effect-of-max_features-number" data-toc-modified-id="Effect-of-max_features-number-7.8.2"><span class="toc-item-num">7.8.2&nbsp;&nbsp;</span>Effect of max_features number</a></span></li><li><span><a href="#Effect-of-number-of-max_leaf_nodes" data-toc-modified-id="Effect-of-number-of-max_leaf_nodes-7.8.3"><span class="toc-item-num">7.8.3&nbsp;&nbsp;</span>Effect of number of max_leaf_nodes</a></span></li><li><span><a href="#Effect-of-min_samples_split" data-toc-modified-id="Effect-of-min_samples_split-7.8.4"><span class="toc-item-num">7.8.4&nbsp;&nbsp;</span>Effect of min_samples_split</a></span></li><li><span><a href="#Final-random-forest-GridSearchCV-tuning" data-toc-modified-id="Final-random-forest-GridSearchCV-tuning-7.8.5"><span class="toc-item-num">7.8.5&nbsp;&nbsp;</span>Final random forest GridSearchCV tuning</a></span></li><li><span><a href="#Random-forest-tuning-using-Hyperopt" data-toc-modified-id="Random-forest-tuning-using-Hyperopt-7.8.6"><span class="toc-item-num">7.8.6&nbsp;&nbsp;</span>Random forest tuning using Hyperopt</a></span></li><li><span><a href="#Random-forest-tuned---reduced-model" data-toc-modified-id="Random-forest-tuned---reduced-model-7.8.7"><span class="toc-item-num">7.8.7&nbsp;&nbsp;</span>Random forest tuned - reduced model</a></span></li></ul></li><li><span><a href="#XGBoost-tuning" data-toc-modified-id="XGBoost-tuning-7.9"><span class="toc-item-num">7.9&nbsp;&nbsp;</span>XGBoost tuning</a></span><ul class="toc-item"><li><span><a href="#Max_depth" data-toc-modified-id="Max_depth-7.9.1"><span class="toc-item-num">7.9.1&nbsp;&nbsp;</span>Max_depth</a></span></li><li><span><a href="#min_child_weight" data-toc-modified-id="min_child_weight-7.9.2"><span class="toc-item-num">7.9.2&nbsp;&nbsp;</span>min_child_weight</a></span></li><li><span><a href="#gamma" data-toc-modified-id="gamma-7.9.3"><span class="toc-item-num">7.9.3&nbsp;&nbsp;</span>gamma</a></span></li><li><span><a href="#XGBoost-manually-tuned-model" data-toc-modified-id="XGBoost-manually-tuned-model-7.9.4"><span class="toc-item-num">7.9.4&nbsp;&nbsp;</span>XGBoost manually tuned model</a></span></li><li><span><a href="#XGBoost-tuning-using-hyperopt" data-toc-modified-id="XGBoost-tuning-using-hyperopt-7.9.5"><span class="toc-item-num">7.9.5&nbsp;&nbsp;</span>XGBoost tuning using hyperopt</a></span></li><li><span><a href="#XGBoost-hyperopt-reduced-model" data-toc-modified-id="XGBoost-hyperopt-reduced-model-7.9.6"><span class="toc-item-num">7.9.6&nbsp;&nbsp;</span>XGBoost hyperopt reduced model</a></span></li></ul></li><li><span><a href="#Is-it-possible-to-build-an-efficient-and-simplified-decision-tree-for-every-rotation_home?" data-toc-modified-id="Is-it-possible-to-build-an-efficient-and-simplified-decision-tree-for-every-rotation_home?-7.10"><span class="toc-item-num">7.10&nbsp;&nbsp;</span>Is it possible to build an efficient and simplified decision tree for every <em>rotation_home</em>?</a></span></li><li><span><a href="#Conclusions-on-region_combination-target" data-toc-modified-id="Conclusions-on-region_combination-target-7.11"><span class="toc-item-num">7.11&nbsp;&nbsp;</span>Conclusions on <em>region_combination</em> target</a></span></li></ul></li><li><span><a href="#Target:-binary_attacker" data-toc-modified-id="Target:-binary_attacker-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Target: binary_attacker</a></span><ul class="toc-item"><li><span><a href="#Label-encoding" data-toc-modified-id="Label-encoding-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>Label encoding</a></span></li><li><span><a href="#Preprocessing-pipeline" data-toc-modified-id="Preprocessing-pipeline-8.2"><span class="toc-item-num">8.2&nbsp;&nbsp;</span>Preprocessing pipeline</a></span></li><li><span><a href="#No-tuning-results" data-toc-modified-id="No-tuning-results-8.3"><span class="toc-item-num">8.3&nbsp;&nbsp;</span>No-tuning results</a></span></li><li><span><a href="#No-tuning-results-(Reduced-model)" data-toc-modified-id="No-tuning-results-(Reduced-model)-8.4"><span class="toc-item-num">8.4&nbsp;&nbsp;</span>No-tuning results (Reduced model)</a></span></li><li><span><a href="#XGBoost-binary-tuning-on-full-set" data-toc-modified-id="XGBoost-binary-tuning-on-full-set-8.5"><span class="toc-item-num">8.5&nbsp;&nbsp;</span>XGBoost binary tuning on full set</a></span><ul class="toc-item"><li><span><a href="#XGBoost-binary-tuning-on-reduced-features-set" data-toc-modified-id="XGBoost-binary-tuning-on-reduced-features-set-8.5.1"><span class="toc-item-num">8.5.1&nbsp;&nbsp;</span>XGBoost binary tuning on reduced features set</a></span></li></ul></li><li><span><a href="#SVC-tuning-on-full-set" data-toc-modified-id="SVC-tuning-on-full-set-8.6"><span class="toc-item-num">8.6&nbsp;&nbsp;</span>SVC tuning on full set</a></span><ul class="toc-item"><li><span><a href="#SVC-on-reduced-set" data-toc-modified-id="SVC-on-reduced-set-8.6.1"><span class="toc-item-num">8.6.1&nbsp;&nbsp;</span>SVC on reduced set</a></span></li></ul></li><li><span><a href="#Logistic-regression-tuning-on-full-set" data-toc-modified-id="Logistic-regression-tuning-on-full-set-8.7"><span class="toc-item-num">8.7&nbsp;&nbsp;</span>Logistic regression tuning on full set</a></span><ul class="toc-item"><li><span><a href="#Logistic-regression-on-reduced-set" data-toc-modified-id="Logistic-regression-on-reduced-set-8.7.1"><span class="toc-item-num">8.7.1&nbsp;&nbsp;</span>Logistic regression on reduced set</a></span></li></ul></li><li><span><a href="#K-Neighbors-tuning-on-full-set" data-toc-modified-id="K-Neighbors-tuning-on-full-set-8.8"><span class="toc-item-num">8.8&nbsp;&nbsp;</span>K-Neighbors tuning on full set</a></span><ul class="toc-item"><li><span><a href="#K-Neighbors-on-reduced-set" data-toc-modified-id="K-Neighbors-on-reduced-set-8.8.1"><span class="toc-item-num">8.8.1&nbsp;&nbsp;</span>K-Neighbors on reduced set</a></span></li></ul></li><li><span><a href="#Ensamble-results" data-toc-modified-id="Ensamble-results-8.9"><span class="toc-item-num">8.9&nbsp;&nbsp;</span>Ensamble results</a></span></li><li><span><a href="#Conclusions-on-binary_attacker-target" data-toc-modified-id="Conclusions-on-binary_attacker-target-8.10"><span class="toc-item-num">8.10&nbsp;&nbsp;</span>Conclusions on <em>binary_attacker</em> target</a></span></li></ul></li><li><span><a href="#Model-interpretation" data-toc-modified-id="Model-interpretation-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Model interpretation</a></span><ul class="toc-item"><li><span><a href="#Global-interpretability" data-toc-modified-id="Global-interpretability-9.1"><span class="toc-item-num">9.1&nbsp;&nbsp;</span>Global interpretability</a></span><ul class="toc-item"><li><span><a href="#Left-(L)" data-toc-modified-id="Left-(L)-9.1.1"><span class="toc-item-num">9.1.1&nbsp;&nbsp;</span>Left (L)</a></span></li><li><span><a href="#Right-(R)" data-toc-modified-id="Right-(R)-9.1.2"><span class="toc-item-num">9.1.2&nbsp;&nbsp;</span>Right (R)</a></span></li><li><span><a href="#Middle-(M)" data-toc-modified-id="Middle-(M)-9.1.3"><span class="toc-item-num">9.1.3&nbsp;&nbsp;</span>Middle (M)</a></span></li></ul></li><li><span><a href="#Partial-dependence-plots" data-toc-modified-id="Partial-dependence-plots-9.2"><span class="toc-item-num">9.2&nbsp;&nbsp;</span>Partial dependence plots</a></span><ul class="toc-item"><li><span><a href="#Importance-of-set_x-in-L/R-decision" data-toc-modified-id="Importance-of-set_x-in-L/R-decision-9.2.1"><span class="toc-item-num">9.2.1&nbsp;&nbsp;</span>Importance of <em>set_x</em> in L/R decision</a></span></li><li><span><a href="#Importance-of-set_y-in-M-decisions" data-toc-modified-id="Importance-of-set_y-in-M-decisions-9.2.2"><span class="toc-item-num">9.2.2&nbsp;&nbsp;</span>Importance of <em>set_y</em> in M decisions</a></span></li></ul></li><li><span><a href="#Local-predictability:-single-side-out-prediction" data-toc-modified-id="Local-predictability:-single-side-out-prediction-9.3"><span class="toc-item-num">9.3&nbsp;&nbsp;</span>Local predictability: single side-out prediction</a></span></li></ul></li><li><span><a href="#General-conclusions" data-toc-modified-id="General-conclusions-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>General conclusions</a></span></li><li><span><a href="#Suggestions-and-next-steps" data-toc-modified-id="Suggestions-and-next-steps-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Suggestions and next steps</a></span></li></ul></div>

# Introduction to the dataset

The dataset was generated using DataVolley DVW files, **kindly provided by César Hernández González (Head Coach of South Korea W's NT and Assistant Coach of Vakifbank Istanbul)**.

A look into five rows of the dataset:

In [ ]:
df_sideout = pd.read_parquet("datasets/conegliano/gold_conegliano.parquet")

In [ ]:
cols = [
    "game_name",
    "opponent_team",
    "rotation_guest",
    "rotation_home",
    "current_game",
    "ScoreScaled",
    "ScoreDelta",
    "reception_quality",
    "reception_x",
    "reception_y",
    "serve_x",
    "call",
    "set_x",
    "set_y",
    "attack_x",
    "sideout_no",
    "F_Block_height",
    "C_Block_height",
    "B_Block_height",
    "attack_quality",
    "SetCode",
    "OppCode",
    "OH1Code",
    "OH2Code",
    "MB1Code",
    "MB2Code",
    "IsSetterBlocking",
    "region_coordinate",
    "attacker",
]

df_sideout = df_sideout[cols]
df_sideout.game_name = df_sideout.game_name.astype(str)
df_sideout.opponent_team = df_sideout.opponent_team.astype(str)
display(df_sideout.sample(5))
print("\n")
print(df_sideout.info())
df_copy = df_sideout.copy()

Each row represents a side-out occurrence and its info.

Let's identify features that are *NOT* useful for the analysis:
- *game_name* (str): unlikely to make a difference, it's just a nice reference if we want to analyze only specific competitions
- *opponent_team* (str): as before, if we want to analyze only a specific opponent
- *current_game* (int): also called set, discrete values from 1 to 5, it is unlikely to provide useful info

Useful features:
- *rotation_guest* (int): Discrete values from 1 to 6. Mostly useful to distinguish rotations with the guest setter blocking or guest opposite blocking
- *rotation_home* (str): Discrete values from 1 to 6. The order of occurrence is always 1-6-5-4-3-2, because of the rule of the volleyball sport. It is encoded as string because sklearn cannot process not-ordered numerical in its LabelEncoder transformer.
- *ScoreScaled* (int): volleyball sets/games end at 25, except the tie-break (5th sets) that ends at 15. Here, the scores in the tie-breaks are multiplied by 1.66 for scaling, so the "heat" of the game is represented somewhat equally for regular sets and tie-breaks
- *ScoreDelta* (int): different in score between Conegliano and the opponent team
- *reception_quality* (char): indication of the quality of the reception. This analysis only includes '#' and '+' passes (the ones of better quality)
- *reception_x* (int): x-coordinate of the reception on the court
- *reception_y* (int): y-coordinate of the reception on the court
- *serve_x* (int): x-coordinate of the serve
- *call* (str): Setter call for the middle blocker
- *set_x* (float): x-coordinate of the setter on the court when setting
- *set_y* (float): y-coordinate of the setter on the court when setting
- *attack_x* (float): x-coordinate of the attacker when hitting the ball
- *sideout_no* (int): number of consecutive side-out occurrences. For example, if equal to 2 it means that Conegliano failed a 1st attempt, and this is the 2nd attempt in a row
- *F_Block_height* (int): block height of the front blocker (usually opposite/setter) in cm
- *B_Block_height* (int): block height of the back blocker (usually outside hitter) in cm
- *C_Block_height* (int): block height of the center blocker (usually middle blocker) in cm
- *IsSetterBlocking* (bool): tells if the opponent is in rotation 2, 3, or 4. It renders the *rotation_guest* attribute less useful
- *attack_quality* (char): outcome of the side-out attack
- *SetCode* (str): code related to the setter present in the court in the side-out rally
- *OppCode* (str): code related to the opposite present in the court in the side-out rally
- *OH1Code* (str): code related to the outside hitter 1 present in the court in the side-out rally
- *OH2Code* (str): code related to the outside hitter 2 present in the court in the side-out rally
- *MB1Code* (str): code related to the middle blocker 1 present in the court in the side-out rally
- *MB2Code* (str): code related to the middle blocker 2 present in the court in the side-out rally

Tentative **target labels**:
- *attacker* (char): identifies the attacker in the rally
- *region_coordinate* (char): identifies the area of the net where the attack occurs

# Data cleaning

Let's start by looking for missing/illegal values, the distribution of the different features, and the kind of information they contain.

## Descriptive statistics

In [ ]:
display(df_sideout.drop(columns=["rotation_guest", "current_game"]).describe().T)

The dataset contains 1521 rows.

Values of -1 for *set_x, set_y* are illegal, indicating missing info.
As this is a relatively small dataset, it is important to carefully inspect all features (luckily they are not too many) to make sure we do not have many outliers or missing values that can alter the predictions.

## set_x
This feature only has 1 missing values (0.06%). We will impute them with the median of the feature during the preprocessing. It looks normally distributed, so it will not need to be transformed.

In [ ]:
print(df_sideout[df_sideout.set_x == -1].game_name.value_counts())
df_sideout.set_x.hist()

## set_y
As for *set_x*, 0.06% of the values are missing and will be imputed. The variable is left-skewed and will need to be transformed with a PowerTransformer or a QuantileTransformer.

In [ ]:
print(df_sideout[df_sideout.set_y == -1].game_name.value_counts())
df_sideout.set_y.hist()

## reception_x
No missing values, the distribution does not have a clear shape, but an almost uniform profile.


In [ ]:
print(df_sideout[df_sideout.reception_x == -1].game_name.value_counts())
df_sideout.reception_x.hist()

## reception_y
No missing values, it appears left-skewed. It needs a PowerTransformer or QuantileTransformer.


In [ ]:
print(df_sideout[df_sideout.reception_y == -1].game_name.value_counts())
df_sideout.reception_y.hist()

## attack_x
No missing values, bimodal distribution. This variable can be binned and used as target label for the setter distribution.

In [ ]:
print(df_sideout[df_sideout.attack_x == -1].game_name.value_counts())
df_sideout.attack_x.hist()

## serve_x
No missing values, bimodal distribution.


In [ ]:
print(df_sideout[df_sideout.serve_x == -1].game_name.value_counts())
df_sideout.serve_x.hist()

## ScoreScaled
No missing values, almost uniform distribution with a small right tail.

In [ ]:
print(df_sideout[df_sideout.ScoreScaled == -1].game_name.value_counts())
df_sideout.ScoreScaled.hist()

## ScoreDelta
No missing values, variable with a light right-skew.

In [ ]:
print(df_sideout[df_sideout.ScoreDelta.isna()].game_name.value_counts())
df_sideout.ScoreDelta.hist()

## sideout_no
The variable is very unbalanced, with about 89% of the occurrences being a first side-out.

In [ ]:
print(df_sideout[df_sideout.sideout_no.isna()].game_name.value_counts())
df_sideout.sideout_no.hist()
print(df_sideout.sideout_no.value_counts(normalize=True))

## block heights
We can notice that *F_Block_height* is the one with the most variability, as often there is a noticeable height difference between opposites and setters (the players blocking on the 'F' side). No missing values.

In [ ]:
import plotly.express as px

fig = px.histogram(
    df_sideout,
    x=["F_Block_height", "C_Block_height", "B_Block_height"],
    nbins=15,
    width=800,
)
fig.show()

## call
There are two under-represented calls: KC and K2. KC can be grouped to K1, as they are functionally similar. K2 will belong to its own under-represented category, but it should not impact the results significantly.

In [ ]:
print(df_sideout.call.value_counts(normalize=True))

import plotly.express as px

fig = px.histogram(df_sideout, x=["call"], width=800)
fig.show()

<img alt="calls" src="images/ensamble_calls.png" width="700"/>

## players codes
Setter is always Wolosz as expected, in the other positions there is a bit of alternance between players. Opposite is usually Egonu (95%), middle blocker 1 is De Krujif (78%) or Butigan (11%), middle blocker 2 is Fahr (57%) or Folie (28%), outside hitter 1 is Sylla (59%) or Mckenzie (25%) or Omoruyi (11%), finally outside hitter 2 is Hill (69%) or McKenzie (24%).

Encoding for these variables can be problematic:
- *SetCode*: it does not add any information and will be dropped
- *OppCode*: it adds only little information, as the opposite was mostly Egonu. It will be dropped
- *MB1Code* and *MB2Code*: I could use an OrdinalEncoder, as the cardinality is not very high. More on this later
- *OH1Code* and *OH2Code*: same as for *MB1Code* and *MB2Code*

In [ ]:
cols = ["SetCode", "OppCode", "OH1Code", "OH2Code", "MB1Code", "MB2Code"]
df_sideout[cols] = df_sideout[cols].astype(str)
print(f"{df_sideout.SetCode.value_counts(normalize=True)}\n")
print(f"{df_sideout.OppCode.value_counts(normalize=True)}\n")
print(f"{df_sideout.MB1Code.value_counts(normalize=True)}\n")
print(f"{df_sideout.MB2Code.value_counts(normalize=True)}\n")
print(f"{df_sideout.OH1Code.value_counts(normalize=True)}\n")
print(f"{df_sideout.OH2Code.value_counts(normalize=True)}\n")

## IsSetterBlocking
Balanced classes for the opponent setter blocking. No missing values.

In [ ]:
print(f"Null values? {df_sideout.IsSetterBlocking.isna().any()}")
print(df_sideout.IsSetterBlocking.value_counts(normalize=True))
df_sideout.rotation_home = df_sideout.rotation_home.astype(str)

# Data preparation and feature engineering
The different features will be properly transformed, encoded, and scaled. Here, the treatment for each feature is explained.

In [ ]:
df_clean = df_sideout.copy()

## serve_x
The variable can be binarized in left-side serve (0) and right-side serve (1) to simplify the feature.

In [ ]:
df_clean.serve_x = np.where(df_clean.serve_x > 50, 1, 0)
df_clean.serve_x.hist()

## sideout_no
I will substitute this feature with a **new binary feature *is_first_sideout***, equal to 1 for True, and 0 for False, hence coalescing all the occurrences of sideout_no > 1 in  *is_first_sideout = False*.

In [ ]:
df_clean = df_clean.assign(is_first_sideout=np.where(df_clean.sideout_no == 1, 1, 0))
print(df_clean.is_first_sideout.value_counts(normalize=True))
df_clean.is_first_sideout.hist()
df_clean = df_clean.drop(columns="sideout_no")

## ScoreScaled
I will coalesce all the right-side tail values to 25 (the end of a set is always an important moment, whether the set ends at 25 or 35).

In [ ]:
df_clean.loc[df_clean.ScoreScaled > 25, "ScoreScaled"] = 25
df_clean.ScoreScaled.hist()

## call
Coalesce KC into K1.

In [ ]:
df_clean.call = df_clean.call.replace({"KC": "K1", "KB": "KF"})
calls_classes = ["K7", "K1", "K2", "KF"]
print(df_clean.call.value_counts(normalize=True))
print(f"Setter classes: {calls_classes}")

## players codes (MB1Code, MB2Code, OH1Code, OH2Code)
For each feature, extract the classes with more than 10% occurrence rate. Use OrdinalEncoder to encode those classes, and coalesce the rest of the unrepresented classes in a single class.

In [ ]:
threshold = 0.10

dt = df_clean.OH1Code.value_counts(normalize=True)
OH1_classes = dt.index.values[dt > threshold].tolist()

dt = df_clean.OH2Code.value_counts(normalize=True)
OH2_classes = dt.index.values[dt > threshold].tolist()

dt = df_clean.MB1Code.value_counts(normalize=True)
MB1_classes = dt.index.values[dt > threshold].tolist()

dt = df_clean.MB2Code.value_counts(normalize=True)
MB2_classes = dt.index.values[dt > threshold].tolist()

print(f"OH1 classes: {OH1_classes}")
print(f"OH2 classes: {OH2_classes}")
print(f"MB1 classes: {MB1_classes}")
print(f"MB2 classes: {MB2_classes}")

## set_y
Use a PowerTransformer for this variable.

In [ ]:
helpclass.display_transformations(df_clean, "set_y", show_normal_test=False)

## ScoreDelta
Use a PowerTransformer.

In [ ]:
helpclass.display_transformations(df_clean, "ScoreDelta", show_normal_test=False)

# Target variables
The goal is to predict who the setter will set to. Some possible variables to look at are:
- *attacker*
- *region_coordinate*
- *binarized_attack*

## attacker
This can be the best possible label for the analysis, as it identifies the attacker that receives the ball: F for front, C for center, B for back, P for pipe, S for setter. However, these are many classes to be predicted with only little data available (about 1500 rows). Moreover, some of the classes (P, S) are under-represented, leading to an unbalanced classification problem.

In [ ]:
df_clean.attacker.value_counts(normalize=True)

![attacker](images/attacker.png)

## region_coordinate
Based on the x-coordinate of the attack (*attack_x*), the attacks have been encoded in only three categories: L for left, M for middle, R for right, identifying which area of the net is targeted by the setter. These labels are much more balanced, and we have only two labels to classify.

In [ ]:
df_clean.region_coordinate.value_counts(normalize=True)

![region_ccordinate](images/region_coordinate.png)

## binarized_attacker
This last target is the simpler:
- 1: the setter is setting in front of her (i.e., *attack_x* >= *set_x*)
- 0: the setter is setting behind her (*attack_x* < *set_x*)

This label is moderately unbalanced, but it could be simpler to analyze.

In [ ]:
df_clean = df_clean.assign(
    binarized_attacker=np.select(
        [df_clean.set_x >= df_clean.attack_x, df_clean.set_x < df_clean.attack_x],
        [0, 1],
        default=-1,
    )
)
print(df_clean.binarized_attacker.value_counts(normalize=True))

![binarized_attacker](images/binary_attacker.png)

## Final choice
Two target labels will be tested: first we will try to predict the set with *region_coordinate*. After that, we will see if the result can be improved by simplifying the classification task to a binary task with the *binarized_attack* label.

# Target: region_combination

In [ ]:
target = "region_coordinate"
df_clean = df_clean.dropna(subset=[target])
print(df_clean[target].value_counts())

In [ ]:
cols = [
    "F_Block_height",
    "C_Block_height",
    "B_Block_height",
    "ScoreDelta",
    "ScoreScaled",
    "reception_x",
    "reception_y",
]
df_clean[cols] = df_clean[cols].astype(float)
df_clean[["serve_x", "is_first_sideout"]] = df_clean[
    ["serve_x", "is_first_sideout"]
].astype(bool)

## Label encoding
Using StratifiedShuffleSplit, with n_splits=1, with 75%-25% train-test split.

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

target = "region_coordinate"
df_temp = df_clean.copy()
df_temp.reset_index(inplace=True, drop=True)

y = df_temp[target]
X = df_temp.drop(columns=target)

sss = StratifiedShuffleSplit(n_splits=1, random_state=42, test_size=0.25)
train_index, test_index = next(sss.split(X, y))

X_train = X.iloc[train_index]
y_train = y.iloc[train_index]
X_test = X.iloc[test_index]
y_test = y.iloc[test_index]

# Training set
enc_target = LabelEncoder()
y_train = enc_target.fit_transform(y_train)
y_test = enc_target.transform(y_test)

print(f"Encoder Classes: {enc_target.classes_}")
print("\nFrequency in stratified shuffle train split:")
print(pd.Series(enc_target.inverse_transform(y_train)).value_counts(normalize=True))

## Preprocessing pipeline

<code>
SetLocationTransformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(missing_values=-1, strategy='median')),
        ("pt", PowerTransformer(standardize=True)),
])

ReceptionTransformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(missing_values=-1, strategy='median')),
])

HeightTransformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy='median')),
])

BinaryTransformer = Pipeline(
    steps=[
        ("enc", OrdinalEncoder(categories=[[False, True]]))
])

SetterCallEncoder = Pipeline([("enc", OrdinalEncoder(categories=[calls_classes], handle_unknown='use_encoded_value', unknown_value=len(calls_classes)))])

TransformerFull = ColumnTransformer(
    transformers=[
        ("ordinal_enc", OrdinalEncoder(categories=[['2', '3', '4', '5', '6', '1']]), ['rotation_home']),
        ("call_enc", SetterCallEncoder, ["call"]),
        ("height_encoder", HeightTransformer, ['F_Block_height', 'B_Block_height', 'C_Block_height']),
        ("reception_loc", ReceptionTransformer, ['reception_x', 'reception_y']),
        ("set_location_enc", SetLocationTransformer, ['set_x', 'set_y']),
        ("binary_imputer", OrdinalEncoder(), ['IsSetterBlocking', 'serve_x', 'is_first_sideout']),
        ("OH1_transformer", OrdinalEncoder(categories=[OH1_classes], handle_unknown='use_encoded_value', unknown_value=len(OH1_classes)), ['OH1Code']),
        ("OH2_transformer", OrdinalEncoder(categories=[OH2_classes], handle_unknown='use_encoded_value', unknown_value=len(OH2_classes)), ['OH2Code']),
        ("MB1_transformer", OrdinalEncoder(categories=[MB1_classes], handle_unknown='use_encoded_value', unknown_value=len(MB1_classes)), ['MB1Code']),
        ("MB2_transformer", OrdinalEncoder(categories=[MB2_classes], handle_unknown='use_encoded_value', unknown_value=len(MB2_classes)), ['MB2Code']),
        ("delta_transformer", PowerTransformer(standardize=True), ['ScoreDelta']),
        ("passthrough", "passthrough", ['ScoreScaled'])
    ],
    remainder="drop",
)

preprocessor_full = Pipeline(
    [
        ('transformer', TransformerFull),
        ('scaler', RobustScaler())
    ]
)
</code>

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PowerTransformer

SetLocationTransformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(missing_values=-1, strategy="median")),
        ("pt", PowerTransformer(standardize=True)),
    ]
)

from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

ReceptionTransformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(missing_values=-1, strategy="median")),
    ]
)

HeightTransformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
    ]
)

BinaryTransformer = Pipeline(
    steps=[("enc", OrdinalEncoder(categories=[[False, True]]))]
)

SetterCallEncoder = Pipeline(
    [
        (
            "enc",
            OrdinalEncoder(
                categories=[calls_classes],
                handle_unknown="use_encoded_value",
                unknown_value=len(calls_classes),
            ),
        )
    ]
)

In [ ]:
from sklearn.preprocessing import RobustScaler

TransformerFull = ColumnTransformer(
    transformers=[
        (
            "ordinal_enc",
            OrdinalEncoder(categories=[["2", "3", "4", "5", "6", "1"]]),
            ["rotation_home"],
        ),
        ("call_enc", SetterCallEncoder, ["call"]),
        (
            "height_encoder",
            HeightTransformer,
            ["F_Block_height", "B_Block_height", "C_Block_height"],
        ),
        ("reception_loc", ReceptionTransformer, ["reception_x", "reception_y"]),
        ("set_location_enc", SetLocationTransformer, ["set_x", "set_y"]),
        (
            "binary_imputer",
            OrdinalEncoder(),
            ["IsSetterBlocking", "serve_x", "is_first_sideout"],
        ),
        (
            "OH1_transformer",
            OrdinalEncoder(
                categories=[OH1_classes],
                handle_unknown="use_encoded_value",
                unknown_value=len(OH1_classes),
            ),
            ["OH1Code"],
        ),
        (
            "OH2_transformer",
            OrdinalEncoder(
                categories=[OH2_classes],
                handle_unknown="use_encoded_value",
                unknown_value=len(OH2_classes),
            ),
            ["OH2Code"],
        ),
        (
            "MB1_transformer",
            OrdinalEncoder(
                categories=[MB1_classes],
                handle_unknown="use_encoded_value",
                unknown_value=len(MB1_classes),
            ),
            ["MB1Code"],
        ),
        (
            "MB2_transformer",
            OrdinalEncoder(
                categories=[MB2_classes],
                handle_unknown="use_encoded_value",
                unknown_value=len(MB2_classes),
            ),
            ["MB2Code"],
        ),
        ("delta_transformer", PowerTransformer(standardize=True), ["ScoreDelta"]),
        ("passthrough", "passthrough", ["ScoreScaled"]),
    ],
    remainder="drop",
)

preprocessor_full = Pipeline(
    [("transformer", TransformerFull), ("scaler", RobustScaler())]
)

## Correlation analysis
As *IsSetterBlocking* is correlated with *F_Block_height*, it is preferable to exclude one of the two variables, as they are likely providing the same info twice.

In [ ]:
def get_features_from_transformer_list(transformers_list):
    features_transformer = []
    for transformer in transformers_list.transformers:
        features_transformer = features_transformer + transformer[2]
    return features_transformer

In [ ]:
import seaborn as sns
from seaborn import heatmap
import matplotlib.pyplot as plt

# sns.set_theme(style="white")
sns.set_theme(style="whitegrid")

features = get_features_from_transformer_list(TransformerFull)
# print(f'Features: {features}')

pp = preprocessor_full
test_df = pd.DataFrame(data=preprocessor_full.fit_transform(X))
test_df.columns = features
corr = test_df.corr()

plt.figure(figsize=(9, 9))
cmap = sns.diverging_palette(220, 0)
heatmap(corr, cmap="coolwarm", center=0, vmin=-1, vmax=1)
plt.show()

In [ ]:
pairs = corr.abs().unstack()
pairs = pairs[pairs < 1]
pairs = pairs.sort_values(kind="quicksort", ascending=False)
print("Most correlated pairs:")
display(pairs.head(5))

In [ ]:
# Remove IsSetterBlocking from pipeline
preprocessor_full.named_steps["transformer"].transformers[5] = (
    "binary_imputer",
    OrdinalEncoder(),
    ["serve_x", "is_first_sideout"],
)

In [ ]:
sklearn.set_config(None, None, None, "diagram")
preprocessor_full

In [ ]:
# Models definition
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    RandomForestClassifier,
    ExtraTreesClassifier,
    GradientBoostingClassifier,
    AdaBoostClassifier,
    HistGradientBoostingClassifier,
)
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import CategoricalNB
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from joblib import dump, load

DT = DecisionTreeClassifier(random_state=42)
RF = RandomForestClassifier(
    oob_score=True, random_state=42, warm_start=True, n_jobs=-1, bootstrap=True
)
ET = ExtraTreesClassifier(oob_score=True, bootstrap=True, random_state=42, n_jobs=-1)
lr = LogisticRegression(
    solver="saga", max_iter=2000, tol=1e-2, dual=False, random_state=42
)
kn = KNeighborsClassifier(n_jobs=-1)
lsvc = LinearSVC(random_state=42, tol=1e-2, max_iter=4000, dual=False)
svc = SVC(random_state=42, tol=1e-2, max_iter=4000, cache_size=1000)
gb = GradientBoostingClassifier(tol=1e-2, random_state=42)
ada = AdaBoostClassifier(random_state=42)
xgbm = xgb.XGBClassifier(use_label_encoder=False, eval_metric="mlogloss")
hgb = HistGradientBoostingClassifier(random_state=42, tol=1e-2)
nb = CategoricalNB()

classifiers = [DT, RF, ET, lr, kn, lsvc, svc, gb, ada, xgbm, hgb]
names = [
    "Decision Tree",
    "Random Forest",
    "Extra Trees",
    "Logistic Regression",
    "K-Neighbors",
    "Linear SVC",
    "SVC",
    "Gradient Boosting",
    "ADABoost",
    "XGBoost",
    "HistGradientBoosting",
]
save_name = ["dt", "rf", "et", "lr", "kn", "lsvc", "svc", "gb", "ada", "xgb", "hgb"]
# params = [DT_params, RF_params, ET_params, lr_params, kn_params, lsvc_params, svc_params, gb_params, ada_params, xgb_params]
parameters = [{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}]

## Model baseline
The baseline is given by assuming the majority class is always the answer. In this case, assuming that the setter choice will always be 0 (i.e., Left side of the net), we would be approximately 36% accurate. **If our model is able to predict with more than 36% accuracy, then we gained some predictive power**.


In [ ]:
print(f"Classes: {enc_target.classes_}")
pd.Series(y_train).value_counts(normalize=True)

## No-tuning results
Let's compare results from different classification models. **We want the model to beat the baseline (accuracy > 36%)**.

As the dataset is small, with a fairly small amount of features (18), I expect simpler models to give the best results. We notice however that the accuracy performance on the training set is 1, while the one on the test set is not: the large difference suggests over-fitting is occurring. We will need to better adjust our models to improve the generalized predictive power (in this case measured with the performance on the test set).

**Out-of-the-box solutions provide values of about 47% accuracy (11% better than the baseline)**.

In [ ]:
importlib.reload(helpclass)
folder = "models/region_coordinate/no_tuning/"
TUNE = False

if TUNE:
    # WARNING: FIT TAKES LONG TIME!
    log_no_tuning_df = helpclass.fit_models(
        X_train,
        y_train,
        X_test,
        y_test,
        preprocessor_full,
        classifiers,
        names,
        save_name,
        folder,
        parameters,
        cross_val_splits=5,
        verbose=False,
    )
else:
    log_no_tuning_df = helpclass.eval_models(
        X_train,
        y_train,
        X_test,
        y_test,
        classifiers,
        names,
        save_name,
        folder,
        verbose=False,
    )

In [ ]:

display(
    log_no_tuning_df[
        ["model", "accuracy", "precision", "recall", "f1", "accuracy_train"]
    ].sort_values(by="accuracy", ascending=False)
)
helpclass.summary_mean_print(log_no_tuning_df)

## Most important features
As the dataset is small, the presence of many useless features can aggravate the over-fitting issue, possibly decreasing the performance of the model. I will use SHAP to identify the most important features for the model prediction. This step would be ideally repeated once models are fine-tuned, to verify the conclusions drawn from the out-of-the-box algorithm used above.

In [ ]:
features = get_features_from_transformer_list(TransformerFull)

In [ ]:

values = []
aux = helpclass.shap_summary_plot_wrapper(
    X_train,
    X_test,
    load("models/region_coordinate/no_tuning/xgb.joblib"),
    features,
    enc_target.classes_,
    model_name="XGBoost",
)
values.append(aux)
aux = helpclass.shap_summary_plot_wrapper(
    X_train,
    X_test,
    load("models/region_coordinate/no_tuning/hgb.joblib"),
    features,
    enc_target.classes_,
    "Hist Gradient Boosting",
)
values.append(aux)
aux = helpclass.shap_summary_plot_wrapper(
    X_train,
    X_test,
    load("models/region_coordinate/no_tuning/rf.joblib"),
    features,
    enc_target.classes_,
    "Random Forest",
)
values.append(aux)

We can point at a few features that do not seem important for any of the first four models: *is_first_sideout*, *MB1Code*, *MB2Code*, *OH1Code*, *OH2Code*. We can check how the models would perform (with no tuning) with a reduced set of features (now 11).

## Reduced set of features
With a reduced set of features, the results did not change significantly. The performance decreased slightly, but the models are simpler. In the next section, we will try to obtain better results by tuning the algorithms on the full set of features.

<code>
TransformerReduced = ColumnTransformer(
    transformers=[
        ("ordinal_enc", OrdinalEncoder(categories=[['2', '3', '4', '5', '6', '1']]), ['rotation_home']),
        ("call_enc", SetterCallEncoder, ["call"]),
        ("height_encoder", HeightTransformer, ['F_Block_height', 'B_Block_height', 'C_Block_height']),
        ("reception_loc", ReceptionTransformer, ['reception_x', 'reception_y']),
        ("set_location_enc", SetLocationTransformer, ['set_x', 'set_y']),
        ("delta_transformer", PowerTransformer(standardize=True), ['ScoreDelta']),
        ("passthrough", "passthrough", ['ScoreScaled'])
    ],
    remainder="drop",
)

preprocessor_reduced = Pipeline(
    [
        ('transformer', TransformerReduced),
        ('scaler', RobustScaler())
    ]
)
</code>

In [ ]:
TransformerReduced = ColumnTransformer(
    transformers=[
        (
            "ordinal_enc",
            OrdinalEncoder(categories=[["2", "3", "4", "5", "6", "1"]]),
            ["rotation_home"],
        ),
        ("call_enc", SetterCallEncoder, ["call"]),
        (
            "height_encoder",
            HeightTransformer,
            ["F_Block_height", "B_Block_height", "C_Block_height"],
        ),
        ("reception_loc", ReceptionTransformer, ["reception_x", "reception_y"]),
        ("set_location_enc", SetLocationTransformer, ["set_x", "set_y"]),
        ("delta_transformer", PowerTransformer(standardize=True), ["ScoreDelta"]),
        ("passthrough", "passthrough", ["ScoreScaled"]),
    ],
    remainder="drop",
)

preprocessor_reduced = Pipeline(
    [("transformer", TransformerReduced), ("scaler", RobustScaler())]
)

In [ ]:
preprocessor_reduced

In [ ]:
folder = "models/region_coordinate/no_tuning_reduced/"
if TUNE:
    log_nt_reduced_df = helpclass.fit_models(
        X_train,
        y_train,
        X_test,
        y_test,
        preprocessor_reduced,
        classifiers,
        names,
        save_name,
        folder,
        parameters,
        cross_val_splits=5,
        verbose=False,
    )
else:
    log_nt_reduced_df = helpclass.eval_models(
        X_train,
        y_train,
        X_test,
        y_test,
        classifiers,
        names,
        save_name,
        folder,
        verbose=False,
    )

In [ ]:
display(
    log_nt_reduced_df[
        ["model", "accuracy", "precision", "recall", "f1", "accuracy_train"]
    ].sort_values(by="accuracy", ascending=False)
)
helpclass.summary_mean_print(log_nt_reduced_df)

## Random forest tuning
As more data is not available to better tune the algorithm, let's manually tune the random forest hyperparameters to try reducing the over-fitting.

### Effect of number of trees
The out-of-bag error is not very affected by the number of trees. A value >200 should suffice.

In [ ]:
folder = "models/tuning_curves/random_forest/"

RF = RandomForestClassifier(oob_score=True, random_state=42, n_jobs=-1, max_samples=0.5)
# RF.set_params(max_features=10, min_samples_split=5)

classifier_pipe = Pipeline(
    steps=[("preprocessor", preprocessor_full), ("classifier", RF)]
)

param = "n_estimators"
plist = np.geomspace(start=50, stop=1200, dtype=int)

if TUNE:
    depth_df = helpclass.get_tuning_curve(
        classifier_pipe, param, plist, X_train, X_test, y_train, y_test
    )
    depth_df.to_csv(f"{folder}{param}.csv")
else:
    depth_df = pd.read_csv(f"{folder}{param}.csv")

import plotly.express as px

fig = px.line(
    depth_df,
    x=param,
    y="oob_err",
    log_x=True,
    width=800,
    title=f"Effect of {param} on out-of-bag error",
)
fig.show()

fig = px.line(
    depth_df,
    x=param,
    y=["train_acc", "test_acc"],
    log_x=True,
    width=800,
    title=f"Effect of {param} on train/test accuracy metric",
)
fig.show()

### Effect of max_features number
The default value for classification is max_features = $\sqrt{\text{features}} = 4$. As it is a small dataset, it can benefit from a larger number of features available for every tree to avoid over-fitting.

In [ ]:
RF = RandomForestClassifier(oob_score=True, random_state=42, n_jobs=-1, max_samples=0.5)
# RF.set_params(max_features=10, min_samples_split=5)

classifier_pipe = Pipeline(
    steps=[("preprocessor", preprocessor_full), ("classifier", RF)]
)

param = "max_features"
plist = [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]

TUNE = False
if TUNE:
    depth_df = helpclass.get_tuning_curve(
        classifier_pipe, param, plist, X_train, X_test, y_train, y_test
    )
    depth_df.to_csv(f"{folder}{param}.csv")
else:
    depth_df = pd.read_csv(f"{folder}{param}.csv")

import plotly.express as px

fig = px.line(
    depth_df,
    x=param,
    y="oob_err",
    width=800,
    title=f"Effect of {param} on out-of-bag error",
)
fig.show()

fig = px.line(
    depth_df,
    x=param,
    y=["train_acc", "test_acc"],
    width=800,
    title=f"Effect of {param} on train/test accuracy metric",
)
fig.show()

### Effect of number of max_leaf_nodes
It is an indirect way to limit the growth of the trees. A lower value would constrain the model more and reduce over-fitting.

In [ ]:
RF = RandomForestClassifier(oob_score=True, random_state=42, n_jobs=-1, max_samples=0.5)

classifier_pipe = Pipeline(
    steps=[("preprocessor", preprocessor_full), ("classifier", RF)]
)

param = "max_leaf_nodes"
plist = [
    5,
    8,
    10,
    12,
    15,
    18,
    20,
    22,
    25,
    27,
    30,
    33,
    35,
    38,
    40,
    43,
    45,
    48,
    50,
    55,
    60,
    65,
]
TUNE = False
if TUNE:
    depth_df = helpclass.get_tuning_curve(
        classifier_pipe, param, plist, X_train, X_test, y_train, y_test
    )
    depth_df.to_csv(f"{folder}{param}.csv")
else:
    depth_df = pd.read_csv(f"{folder}{param}.csv")

import plotly.express as px

fig = px.line(
    depth_df,
    x=param,
    y="oob_err",
    width=800,
    title=f"Effect of {param} on out-of-bag error",
)
fig.show()

fig = px.line(
    depth_df,
    x=param,
    y=["train_acc", "test_acc"],
    width=800,
    title=f"Effect of {param} on train/test accuracy metric",
)
fig.show()

### Effect of min_samples_split
Larger values decrease over-fitting, by limiting the number of nodes that can be further split based on their population.

In [ ]:
RF = RandomForestClassifier(oob_score=True, random_state=42, n_jobs=-1, max_samples=0.5)

classifier_pipe = Pipeline(
    steps=[("preprocessor", preprocessor_full), ("classifier", RF)]
)

param = "min_samples_split"
plist = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 20, 25, 30, 35, 40, 45, 50]

TUNE = False
if TUNE:
    depth_df = helpclass.get_tuning_curve(
        classifier_pipe, param, plist, X_train, X_test, y_train, y_test
    )
    depth_df.to_csv(f"{folder}{param}.csv")
else:
    depth_df = pd.read_csv(f"{folder}{param}.csv")

import plotly.express as px

fig = px.line(
    depth_df,
    x=param,
    y="oob_err",
    width=800,
    title=f"Effect of {param} on out-of-bag error",
)
fig.show()

fig = px.line(
    depth_df,
    x=param,
    y=["train_acc", "test_acc"],
    width=800,
    title=f"Effect of {param} on train/test accuracy metric",
)
fig.show()

### Final random forest GridSearchCV tuning
A grid search among hyperparameters is performed based on the most promising hyperparameters values that reduced the out-of-bag error in the tests above.
Compared to the initial no-tune results (Random Forest, accuracy=0.467192, precision=0.470319, recall=0.463167, f1=0.464306, accuracy_train=1.000000) the model scores slightly worse (accuracy = 0.46982, f1=0.459596) but with 0.78 accuracy for the train set (this could be a good sign in terms of reducing over-fitting).

In [ ]:
from helpclass import report_model_performance

# putting it all together
RF = RandomForestClassifier(random_state=42, n_jobs=-1)

folder = "models/region_coordinate/tuned/"
model = "rf"
name = "Random Forest"

if TUNE:
    params = {
        "classifier__n_estimators": [170, 250],
        "classifier__max_features": [5, 10, 13],
        # 'classifier__max_depth' : [6, 7, 8],
        "classifier__criterion": ["gini", "entropy"],
        "classifier__min_samples_split": [4, 7, 10],
        "classifier__bootstrap": [True, False],
        "classifier__max_leaf_nodes": [30, 43, None],
    }

    classifier_pipe = Pipeline(
        steps=[("preprocessor", preprocessor_full), ("classifier", RF)]
    )

    CV = GridSearchCV(classifier_pipe, params, n_jobs=-1, cv=5)
    CV.fit(X_train, y_train)
    print(CV.best_params_)
    report_model_performance(
        X_train,
        X_test,
        y_train,
        y_test,
        CV.best_estimator_,
        name,
        param_grid=params,
        verbose=True,
    )
    dump(CV.best_estimator_, f"{folder}{model}.joblib")
else:
    loaded_model = load(f"{folder}{model}.joblib")
    print(loaded_model.named_steps["classifier"])
    print("\n")
    helpclass.report_model_performance(
        X_train, X_test, y_train, y_test, loaded_model, name, verbose=True
    )

### Random forest tuning using Hyperopt
[Hyperopt](http://hyperopt.github.io/hyperopt/getting-started/overview/) allows for the search of different hyperparameters that minimize a given function (in this case, accuracy). In this case, the results using Hyperopt have been lower than the ones with manual tuning. It is possible that more Hyperopt iterations are needed, or that the model above "got lucky" on the test set, but would not perform as well on different test set.

In [ ]:
from hyperopt import hp

space = {
    "max_features": hp.uniform("max_features", 4, 16),
    "criterion": hp.choice("criterion", ["gini", "entropy"]),
    "min_samples_split": hp.quniform("min_samples_split", 4, 15, 1),
    "max_leaf_nodes": hp.uniform("max_leaf_nodes", 30, 45),
    "n_estimators": 200,
}

from hyperopt import Trials, fmin, tpe

X_tr_processed = preprocessor_full.fit_transform(X_train)
X_ts_preprocessed = preprocessor_full.transform(X_test)


def objective_rf(explore_space):
    clf = RandomForestClassifier(
        oob_score=True,
        random_state=42,
        warm_start=False,
        n_jobs=-1,
        max_samples=0.5,
        max_features=int(explore_space["max_features"]),
        n_estimators=int(explore_space["n_estimators"]),
        bootstrap=True,
        criterion=explore_space["criterion"],
        max_leaf_nodes=int(explore_space["max_leaf_nodes"]),
        min_samples_split=int(explore_space["min_samples_split"]),
    )

    clf.fit(X_tr_processed, y_train)
    scores = cross_val_score(clf, X_tr_processed, y_train, scoring="accuracy", cv=5)
    score_avg = scores.mean()

    # print (f"5-CV mean SCORE: {score_avg}")
    return {"loss": -score_avg, "status": STATUS_OK}

In [ ]:
if TUNE:
    trials = Trials()
    best_hyperparams = fmin(
        fn=objective_rf, space=space, algo=tpe.suggest, max_evals=200, trials=trials
    )
    print("The best hyperparameters are : ", "\n")
    print(best_hyperparams)

In [ ]:

folder = "models/region_coordinate/tuned/"
model = "rf_hyperopt"
name = "Random Forest"

if TUNE:
    # putting it all together
    RF = RandomForestClassifier(random_state=42, n_jobs=-1, max_samples=0.5)

    params = {
        "classifier__n_estimators": [200],
        "classifier__max_features": [11],
        "classifier__criterion": ["entropy"],
        "classifier__min_samples_split": [8],
        "classifier__bootstrap": [False],
        "classifier__max_leaf_nodes": [35],
    }

    classifier_pipe = Pipeline(
        steps=[("preprocessor", preprocessor_full), ("classifier", RF)]
    )

    CV = GridSearchCV(classifier_pipe, params, n_jobs=-1, cv=5)
    CV.fit(X_train, y_train)
    print(CV.best_params_)
    print(CV.scorer_)
    helpclass.report_model_performance(
        X_train,
        X_test,
        y_train,
        y_test,
        CV.best_estimator_,
        name,
        param_grid=params,
        verbose=True,
    )
    dump(CV.best_estimator_, f"{folder}{model}.joblib")
else:
    loaded_model = load(f"{folder}{model}.joblib")
    print(loaded_model.named_steps["classifier"])
    print("\n")
    report_model_performance(
        X_train, X_test, y_train, y_test, loaded_model, name, verbose=True
    )

### Random forest tuned - reduced model
With a reduced model (model with reduced numbers of features) the performance loss is very small. The performance of the tuned model is better than the no-tuning alternative (+1.8% accuracy and +1.5% f1-score).

In [ ]:
# putting it all together

folder = "models/region_coordinate/tuned/"
model = "rf_reduced_features"
name = "Random Forest"

if TUNE:
    RF = RandomForestClassifier(random_state=42, n_jobs=-1)

    params = {
        "classifier__n_estimators": [170, 250],
        "classifier__max_features": [5, None],
        # 'classifier__max_depth' : [6, 7, 8],
        "classifier__criterion": ["gini", "entropy"],
        "classifier__min_samples_split": [4, 7, 10],
        "classifier__bootstrap": [True, False],
        "classifier__max_leaf_nodes": [15, 30, 43],
    }

    classifier_pipe = Pipeline(
        steps=[("preprocessor", preprocessor_reduced), ("classifier", RF)]
    )

    CV = GridSearchCV(classifier_pipe, params, n_jobs=-1, cv=5)
    CV.fit(X_train, y_train)
    print(CV.best_params_)
    report_model_performance(
        X_train,
        X_test,
        y_train,
        y_test,
        CV.best_estimator_,
        name,
        param_grid=params,
        verbose=True,
    )
    dump(CV.best_estimator_, f"{folder}{model}.joblib")

else:
    loaded_model = load(f"{folder}{model}.joblib")
    print(loaded_model.named_steps["classifier"])
    print("\n")
    report_model_performance(
        X_train, X_test, y_train, y_test, loaded_model, name, verbose=True
    )

## XGBoost tuning
First, let's evaluate the effect of different parameters on the train and test performance. Later, a search with GridSearchCV will be run to find the hyperparameters that give the best results. Then, Hyperopt will be used and the results compared.

XGBoost offers several tuning parameters. As the dataset is small, with high risk of over-fitting, we will need to impose some regularization to have a model that can generalize well to new data. Let's analyze the effect of different hyperparameters on our metrics.

### Max_depth
Increasing max_depth reduces bias but increases variance. Lower values indicate lower complexity, and they are preferrable to face the over-fitting issue.

In [ ]:
import xgboost as xgb

folder = "models/tuning_curves/xgboost/"
classifier_pipe = Pipeline(
    steps=[
        ("preprocessor", preprocessor_full),
        (
            "classifier",
            xgb.XGBClassifier(
                use_label_encoder=False, eval_metric="mlogloss", subsample=0.5
            ),
        ),
    ]
)

param = "max_depth"
plist = [2, 3, 4, 5, 6, 7, 8, 9, 10]


TUNE = False
if TUNE:
    depth_df = helpclass.get_tuning_curve(
        classifier_pipe, param, plist, X_train, X_test, y_train, y_test, skip_oob=True
    )
    depth_df.to_csv(f"{folder}{param}.csv")
else:
    depth_df = pd.read_csv(f"{folder}{param}.csv")

import plotly.express as px

fig = px.line(
    depth_df,
    x=param,
    y=["train_acc", "test_acc"],
    width=800,
    title=f"Effect of {param} on train/test accuracy metric",
)
fig.show()

### min_child_weight
It indirectly impact the complexity of the trees. Lower values allow more node splits to take place, increasing variance and reducing bias, but they may lead to over-fitting issues.

In [ ]:
classifier_pipe = Pipeline(
    steps=[
        ("preprocessor", preprocessor_full),
        (
            "classifier",
            xgb.XGBClassifier(
                use_label_encoder=False, eval_metric="mlogloss", subsample=0.5
            ),
        ),
    ]
)

param = "min_child_weight"
plist = np.geomspace(1, 20, num=30)

TUNE = False
if TUNE:
    depth_df = helpclass.get_tuning_curve(
        classifier_pipe, param, plist, X_train, X_test, y_train, y_test, skip_oob=True
    )
    depth_df.to_csv(f"{folder}{param}.csv")
else:
    depth_df = pd.read_csv(f"{folder}{param}.csv")

import plotly.express as px

fig = px.line(
    depth_df,
    x=param,
    y=["train_acc", "test_acc"],
    width=800,
    title=f"Effect of {param} on train/test accuracy metric",
)
fig.show()

### gamma
Higher values of gamma are more conservative (i.e., they inhibit further partitioning of leaf nodes). Larger values of gamma might reduce over-fitting.

In [ ]:
classifier_pipe = Pipeline(
    steps=[
        ("preprocessor", preprocessor_full),
        (
            "classifier",
            xgb.XGBClassifier(
                use_label_encoder=False, eval_metric="mlogloss", subsample=0.5
            ),
        ),
    ]
)

param = "gamma"
plist = np.geomspace(1e-4, 5, num=30)

TUNE = False
if TUNE:
    depth_df = helpclass.get_tuning_curve(
        classifier_pipe, param, plist, X_train, X_test, y_train, y_test, skip_oob=True
    )
    depth_df.to_csv(f"{folder}{param}.csv")
else:
    depth_df = pd.read_csv(f"{folder}{param}.csv")

import plotly.express as px

fig = px.line(
    depth_df,
    x=param,
    y=["train_acc", "test_acc"],
    width=800,
    title=f"Effect of {param} on train/test accuracy metric",
)
fig.show()

### XGBoost manually tuned model
- With GridSearchCV, investigate the number of estimators and learning rate.

In [ ]:
# 1. Look for number of trees
# Following: https://bradleyboehmke.github.io/HOML/gbm.html#xgboost

import xgboost as xgb

folder = "models/region_coordinate/tuned/"
model = "xgb"
name = "XGBoost"

TUNE = False
if TUNE:
    param_grid = {
        "classifier__n_estimators": [100, 180, 300, 500],
        "classifier__eta": [1e-3, 0.01, 0.05],
    }

    complete_pipe = Pipeline(
        steps=[
            ("preprocessor", preprocessor_full),
            (
                "classifier",
                xgb.XGBClassifier(
                    use_label_encoder=False, subsample=0.7, eval_metric="mlogloss"
                ),
            ),
        ]
    )
    CV = GridSearchCV(complete_pipe, param_grid, n_jobs=-1, cv=5)
    CV.fit(
        X_train,
        y_train,
    )


    print(CV.best_params_)
    print(CV.best_score_)
    report_model_performance(
        X_train,
        X_test,
        y_train,
        y_test,
        CV.best_estimator_,
        name,
        param_grid=param_grid,
        verbose=True,
    )

- Then, change tree-specific parameters, such as the max_depth and min_child_weight.

In [ ]:
# 2. Tree-specific parameters
# Following: https://bradleyboehmke.github.io/HOML/gbm.html#xgboost

import xgboost as xgb

folder = "models/region_coordinate/tuned/"
model = "xgb"
name = "XGBoost"
if TUNE:
    param_grid = {
        "classifier__n_estimators": [300],
        "classifier__eta": [0.01],
        "classifier__max_depth": [5, 6, 7],
        "classifier__min_child_weight": [3, 4, 6, 8],
    }

    complete_pipe = Pipeline(
        steps=[
            ("preprocessor", preprocessor_full),
            (
                "classifier",
                xgb.XGBClassifier(
                    use_label_encoder=False, subsample=0.7, eval_metric="mlogloss"
                ),
            ),
        ]
    )
    CV = GridSearchCV(complete_pipe, param_grid, n_jobs=-1, cv=5)
    CV.fit(X_train, y_train)

    print(CV.best_params_)
    print(CV.best_score_)
    report_model_performance(
        X_train,
        X_test,
        y_train,
        y_test,
        CV.best_estimator_,
        name,
        param_grid=params,
        verbose=True,
    )

- Introduce the gamma hyperparameter.

In [ ]:
# 3a. Regularization
# Following: https://bradleyboehmke.github.io/HOML/gbm.html#xgboost

import xgboost as xgb

folder = "models/region_coordinate/tuned/"
model = "xgb"
name = "XGBoost"
if TUNE:
    param_grid = {
        "classifier__n_estimators": [300],
        "classifier__eta": [0.01],
        "classifier__max_depth": [6],
        "classifier__min_child_weight": [3],
        "classifier__gamma": [0, 0.25, 0.5, 0.75, 1],
    }

    complete_pipe = Pipeline(
        steps=[
            ("preprocessor", preprocessor_full),
            (
                "classifier",
                xgb.XGBClassifier(
                    use_label_encoder=False, subsample=0.7, eval_metric="mlogloss"
                ),
            ),
        ]
    )
    CV = GridSearchCV(complete_pipe, param_grid, n_jobs=-1, cv=5)
    CV.fit(X_train, y_train)

    print(CV.best_params_)
    print(CV.best_score_)
    report_model_performance(
        X_train,
        X_test,
        y_train,
        y_test,
        CV.best_estimator_,
        name,
        param_grid=params,
        verbose=True,
    )

- Introduce lambda and alpha regularization.

In [ ]:
# 3b. Regularization
# Following: https://bradleyboehmke.github.io/HOML/gbm.html#xgboost

import xgboost as xgb

folder = "models/region_coordinate/tuned/"
model = "xgb"
name = "XGBoost"
if TUNE:
    param_grid = {
        "classifier__n_estimators": [300],
        "classifier__eta": [0.01],
        "classifier__max_depth": [6],
        "classifier__min_child_weight": [3],
        "classifier__gamma": [0.5],
        "classifier__lambda": [1, 1.25, 1.5, 1.75, 2],
        "classifier__alpha": [0, 0.25, 0.5, 1, 1.5],
    }

    complete_pipe = Pipeline(
        steps=[
            ("preprocessor", preprocessor_full),
            (
                "classifier",
                xgb.XGBClassifier(
                    use_label_encoder=False, subsample=0.7, eval_metric="mlogloss"
                ),
            ),
        ]
    )
    CV = GridSearchCV(complete_pipe, param_grid, n_jobs=-1, cv=5)
    CV.fit(X_train, y_train)

    print(CV.best_params_)
    print(CV.best_score_)
    report_model_performance(
        X_train,
        X_test,
        y_train,
        y_test,
        CV.best_estimator_,
        name,
        param_grid=params,
        verbose=True,
    )

- Review the learning rate with the new hyperparameters determined above.

In [ ]:
# 4. Review learning rate
# Following: https://bradleyboehmke.github.io/HOML/gbm.html#xgboost

import xgboost as xgb

folder = "models/region_coordinate/tuned/"
model = "xgb"
name = "XGBoost"
if TUNE:
    param_grid = {
        "classifier__n_estimators": [300],
        "classifier__eta": [1e-3, 5e-3, 1e-2, 5e-2, 1e-1],
        "classifier__max_depth": [6],
        "classifier__min_child_weight": [3],
        "classifier__gamma": [0.5],
        "classifier__lambda": [1],
        "classifier__alpha": [1],
    }

    complete_pipe = Pipeline(
        steps=[
            ("preprocessor", preprocessor_full),
            (
                "classifier",
                xgb.XGBClassifier(
                    use_label_encoder=False, subsample=0.8, eval_metric="mlogloss"
                ),
            ),
        ]
    )
    CV = GridSearchCV(complete_pipe, param_grid, n_jobs=-1, cv=5)
    CV.fit(X_train, y_train)

    print(CV.best_params_)
    print(CV.best_score_)
    report_model_performance(
        X_train,
        X_test,
        y_train,
        y_test,
        CV.best_estimator_,
        name,
        param_grid=params,
        verbose=True,
    )

# {'classifier__alpha': 0, 'classifier__eta': 0.01, 'classifier__gamma': 1, 'classifier__lambda': 1, 'classifier__max_depth': 6, 'classifier__min_child_weight': 8, 'classifier__n_estimators': 300}
# 0.4973684210526315
# XGBoost
# Accuracy score on test set: 0.46194225721784776
# F1-score on test set: 0.45594319100433883

- Now finalize the classifier by tuning it on the complete training set with 5-fold cross validation.

In [ ]:
import xgboost as xgb

folder = "models/region_coordinate/tuned/"
model = "xgb"
name = "XGBoost"

TUNE = False
if TUNE:
    param_grid = {
        "classifier__subsample": [0.7],
        "classifier__n_estimators": [300],
        "classifier__max_depth": [6],
        "classifier__min_child_weight": [3, 8],
        "classifier__gamma": [0.5, 1],
        "classifier__eta": [0.01],
        "classifier__lambda": [1],
        "classifier__alpha": [0, 1],
    }

    complete_pipe = Pipeline(
        steps=[
            ("preprocessor", preprocessor_full),
            (
                "classifier",
                xgb.XGBClassifier(use_label_encoder=False, eval_metric="mlogloss"),
            ),
        ]
    )
    CV = GridSearchCV(complete_pipe, param_grid, n_jobs=-1, cv=5)
    CV.fit(X_train, y_train)

    print(CV.best_params_)
    dump(CV.best_estimator_, f"{folder}{model}.joblib")
    report_model_performance(
        X_train,
        X_test,
        y_train,
        y_test,
        CV.best_estimator_,
        name,
        param_grid=params,
        verbose=True,
    )
else:
    loaded_model = load(f"{folder}{model}.joblib")
    print(loaded_model.named_steps["classifier"])
    print("\n")
    report_model_performance(
        X_train, X_test, y_train, y_test, loaded_model, name, verbose=True
    )

### XGBoost tuning using hyperopt

In [ ]:
from hyperopt import hp

space = {
    "max_depth": hp.quniform("max_depth", 4, 8, 1),
    "gamma": hp.uniform("gamma", 0, 3),
    "eta": hp.uniform("eta", 1e-4, 0.1),
    "reg_alpha": hp.uniform("reg_alpha", 0, 3),
    "reg_lambda": hp.uniform("reg_lambda", 1, 4),
    "min_child_weight": hp.uniform("min_child_weight", 1.5, 10),
    "n_estimators": hp.quniform("n_estimators", 50, 800, 1),
    "seed": 42,
}

In [ ]:
from hyperopt import Trials, fmin, tpe

X_tr_processed = preprocessor_full.fit_transform(X_train)
X_ts_preprocessed = preprocessor_full.transform(X_test)


def objective_xgb(explore_space):
    clf = xgb.XGBClassifier(
        use_label_encoder=False,
        subsample=0.5,
        eval_metric="mlogloss",
        n_estimators=int(explore_space["n_estimators"]),
        max_depth=int(explore_space["max_depth"]),
        gamma=explore_space["gamma"],
        eta=explore_space["eta"],
        reg_alpha=explore_space["reg_alpha"],
        min_child_weight=explore_space["min_child_weight"],
        reg_lambda=explore_space["reg_lambda"],
    )

    evaluation = [(X_tr_processed, y_train), (X_ts_preprocessed, y_test)]

    clf.fit(
        X_tr_processed,
        y_train,
        eval_set=evaluation,
        eval_metric="mlogloss",
        early_stopping_rounds=50,
        verbose=False,
    )
    scores = cross_val_score(clf, X_tr_processed, y_train, scoring="f1_macro")
    score_avg = scores.mean()

    # print (f"5-CV f1 SCORE: {score_avg}")
    return {"loss": -score_avg, "status": STATUS_OK}

In [ ]:
if TUNE:
    trials = Trials()
    best_hyperparams = fmin(
        fn=objective_xgb, space=space, algo=tpe.suggest, max_evals=200, trials=trials
    )
    print("The best hyperparameters are : ", "\n")
    print(best_hyperparams)

Once a good set of hyperparameters has been found with Hyperopt, retrain a model using those parameters on the full training set with 5-fold cross-validation.

In [ ]:
import xgboost as xgb

folder = "models/region_coordinate/tuned/"
model = "xgb_hyperopt"
name = "XGBoost"

if TUNE:
    param_grid = {
        "classifier__subsample": [0.7],
        "classifier__n_estimators": [309],
        "classifier__max_depth": [5],
        "classifier__min_child_weight": [2.548],
        "classifier__gamma": [0.1789],
        "classifier__eta": [0.0158],
        "classifier__lambda": [1.522],
        "classifier__alpha": [0.285],
        # 'classifier__subsample': [0.7],
        # 'classifier__n_estimators': [300],
        # 'classifier__max_depth': [6],
        # 'classifier__min_child_weight': [4.64],
        # 'classifier__gamma': [0.85],
        # 'classifier__eta': [0.0106],
        # 'classifier__lambda': [2.64],
        # 'classifier__alpha': [0.836],
    }

    complete_pipe = Pipeline(
        steps=[
            ("preprocessor", preprocessor_full),
            (
                "classifier",
                xgb.XGBClassifier(use_label_encoder=False, eval_metric="mlogloss"),
            ),
        ]
    )
    CV = GridSearchCV(complete_pipe, param_grid, n_jobs=-1, cv=5)
    CV.fit(X_train, y_train)

    print(CV.best_params_)
    print(CV.best_score_)
    dump(CV.best_estimator_, f"{folder}{model}.joblib")
    report_model_performance(
        X_train,
        X_test,
        y_train,
        y_test,
        CV.best_estimator_,
        name,
        param_grid=param_grid,
        verbose=True,
    )
else:
    loaded_model = load(f"{folder}{model}.joblib")
    print(loaded_model.named_steps["classifier"])
    print("\n")
    report_model_performance(
        X_train, X_test, y_train, y_test, loaded_model, name, verbose=True
    )

### XGBoost hyperopt reduced model
We could repeat the procedure on the reduced model. Results:
- XGBoost full model: 0.469 accuracy, 0.463 f1-score
- XGBoost reduced model: 0.459 accuracy, 0.458 f1-score

In [ ]:
X_tr_processed = preprocessor_reduced.fit_transform(X_train)
X_ts_preprocessed = preprocessor_reduced.transform(X_test)


def objective_xgb(explore_space):
    clf = xgb.XGBClassifier(
        use_label_encoder=False,
        subsample=0.5,
        eval_metric="mlogloss",
        n_estimators=int(explore_space["n_estimators"]),
        max_depth=int(explore_space["max_depth"]),
        gamma=explore_space["gamma"],
        eta=explore_space["eta"],
        reg_alpha=explore_space["reg_alpha"],
        min_child_weight=explore_space["min_child_weight"],
        reg_lambda=explore_space["reg_lambda"],
    )

    evaluation = [(X_tr_processed, y_train), (X_ts_preprocessed, y_test)]

    clf.fit(
        X_tr_processed,
        y_train,
        eval_set=evaluation,
        eval_metric="mlogloss",
        early_stopping_rounds=30,
        verbose=False,
    )
    scores = cross_val_score(clf, X_tr_processed, y_train, scoring="f1_macro")
    scores_avg = scores.mean()

    print(f"5-CV avg SCORE: {scores_avg}")
    return {"loss": -scores_avg, "status": STATUS_OK}

In [ ]:
if TUNE:
    trials = Trials()
    best_hyperparams = fmin(
        fn=objective_xgb, space=space, algo=tpe.suggest, max_evals=100, trials=trials
    )
    print("The best hyperparameters are : ", "\n")
    print(best_hyperparams)

In [ ]:
import xgboost as xgb

folder = "models/region_coordinate/tuned/"
model = "xgb_hyperopt_reduced"
name = "XGBoost"

if TUNE:
    param_grid = {
        "classifier__subsample": [0.7],
        "classifier__n_estimators": [287],
        "classifier__max_depth": [6],
        "classifier__min_child_weight": [8.344],
        "classifier__gamma": [0.837],
        "classifier__eta": [0.051],
        "classifier__lambda": [3.093],
        "classifier__alpha": [0.324],
        # 'classifier__subsample': [0.7],
        # 'classifier__n_estimators': [300],
        # 'classifier__max_depth': [8],
        # 'classifier__min_child_weight': [4.96],
        # 'classifier__gamma': [0.844],
        # 'classifier__eta': [0.029],
        # 'classifier__lambda': [2.92],
        # 'classifier__alpha': [1.184],
    }

    complete_pipe = Pipeline(
        steps=[
            ("preprocessor", preprocessor_reduced),
            (
                "classifier",
                xgb.XGBClassifier(use_label_encoder=False, eval_metric="mlogloss"),
            ),
        ]
    )
    CV = GridSearchCV(complete_pipe, param_grid, n_jobs=-1, cv=5)
    CV.fit(X_train, y_train)

    print(CV.best_params_)
    print(CV.best_score_)
    dump(CV.best_estimator_, f"{folder}{model}.joblib")
    report_model_performance(
        X_train,
        X_test,
        y_train,
        y_test,
        CV.best_estimator_,
        name,
        param_grid=param_grid,
        verbose=True,
    )
else:
    loaded_model = load(f"{folder}{model}.joblib")
    print(loaded_model.named_steps["classifier"])
    print("\n")
    report_model_performance(
        X_train, X_test, y_train, y_test, loaded_model, name, verbose=True
    )

## Is it possible to build an efficient and simplified decision tree for every *rotation_home*?
Unfortunately, **the simplified decision tree (max depth = 3) accuracy (33%) is lower than the baseline (40%)** for rotation 6 (sample of *rotation_home*). It is unlikely that this method can be used without additional data-points.

In [ ]:
target = "region_coordinate"
selected_rotation = "6"
df_temp6 = df_clean.copy()
df_temp6 = df_temp6[df_temp6.rotation_home == selected_rotation]
df_temp6.reset_index(inplace=True, drop=True)

print(f"Number of rallies for rotation {selected_rotation}: {len(df_temp6)}")

y6 = df_temp6[target]
X6 = df_temp6.drop(columns=target)

sss = StratifiedShuffleSplit(n_splits=1, random_state=42, test_size=0.5)
train_index6, test_index6 = next(sss.split(X6, y6))

X_train6 = X6.iloc[train_index6]
y_train6 = y6.iloc[train_index6]
X_test6 = X6.iloc[test_index6]
y_test6 = y6.iloc[test_index6]

# Training set
enc_target6 = LabelEncoder()
print(f"\nDistribution in train set:\n{y_train6.value_counts(normalize=True)}")
print(
    f"\nBaseline accuracy for rotation {selected_rotation}: {max(y_train6.value_counts(normalize=True))*100:.1f}%"
)
y_train6 = enc_target6.fit_transform(y_train6)
y_test6 = enc_target6.transform(y_test6)

In [ ]:
from hyperopt import hp

space = {
    "max_depth": hp.quniform("max_depth", 2, 3, 1),
    "max_features": hp.uniform("max_features", 2, 8),
    # 'max_leaf_nodes' : hp.quniform('max_leaf_nodes', 5, 25, 1),
    "min_samples_split": hp.quniform("min_samples_split", 5, 10, 1),
    # 'min_samples_leaf': hp.quniform("min_samples_leaf", 1, 5, 1),
}

In [ ]:
X_tr_processed6 = preprocessor_reduced.fit_transform(X_train6)


def objective_dt(explore_space):
    clf = DecisionTreeClassifier(
        random_state=42,
        max_depth=int(explore_space["max_depth"]),
        ccp_alpha=0,
        # max_leaf_nodes=int(explore_space['max_leaf_nodes']),
        min_samples_split=int(explore_space["min_samples_split"]),
        # min_samples_leaf=int(explore_space['min_samples_leaf']),
        max_features=int(explore_space["max_features"]),
    )

    clf.fit(X_tr_processed6, y_train6)
    scores = cross_val_score(clf, X_tr_processed6, y_train6, scoring="f1_macro", cv=5)
    scores_avg = scores.mean()

    # print (f"5-CV avg SCORE: {scores_avg}")
    return {"loss": -scores_avg, "status": STATUS_OK}


TUNE = False
if TUNE:
    trials = Trials()
    best_hyperparams = fmin(
        fn=objective_dt, space=space, algo=tpe.suggest, max_evals=250, trials=trials
    )
    print("The best hyperparameters are : ", "\n")
    print(best_hyperparams)

In [ ]:
folder = "models/region_coordinate/tuned/"
model = "dt_hyperopt_reduced"
name = "Decision Tree"

if TUNE:
    param_grid = {
        "classifier__ccp_alpha": [0],
        "classifier__max_depth": [3],
        "classifier__max_features": [5],
        # 'classifier__max_leaf_nodes': [25],
        # 'classifier__min_samples_leaf': [4],
        "classifier__min_samples_split": [5],
    }

    complete_pipe = Pipeline(
        steps=[
            ("preprocessor", preprocessor_reduced),
            ("classifier", DecisionTreeClassifier(random_state=42)),
        ]
    )
    CV = GridSearchCV(complete_pipe, param_grid, n_jobs=-1, cv=5)
    CV.fit(X_train6, y_train6)

    print(CV.best_params_)
    print(CV.best_score_)
    dump(CV.best_estimator_, f"{folder}{model}.joblib")
    report_model_performance(
        X_train6,
        X_test6,
        y_train6,
        y_test6,
        CV.best_estimator_,
        name,
        param_grid=param_grid,
        verbose=True,
    )
else:
    loaded_model = load(f"{folder}{model}.joblib")
    print(loaded_model.named_steps["classifier"])
    print("\n")
    report_model_performance(
        X_train6, X_test6, y_train6, y_test6, loaded_model, name, verbose=True
    )

In [ ]:


if TUNE:
    from sklearn import tree

    if loaded_model:
        dot_data = tree.export_graphviz(
            loaded_model.named_steps["classifier"],
            out_file="images/tree.dot",
            feature_names=get_features_from_transformer_list(
                loaded_model.named_steps["preprocessor"].named_steps["transformer"]
            ),
            class_names=enc_target.classes_,
            filled=True,
            rounded=True,
            special_characters=True,
        )
        # graph = graphviz.Source(dot_data)

<img alt="dt" src="images/dt.png" width="900"/>

## Conclusions on *region_combination* target
**Using *region_combination* as a target, we were able to achieve 47% accuracy in prediction, up from the 36% baseline**.

The result is **encouraging, but not too impressive**. Possible improvements might come from the availability of **more data**, or using **different features** that were not included in this model (more about this in the general conclusions at the end of this work).

It is worth noticing that if the process was completely random in the three classifications labels, then it would be impossible to predict the result with more than 1/3 accuracy. The results above show that there is a level of prediction possible, but the mis-classifications could be due to insufficient data, inappropriate features, or also the **random nature of the process**.

Next, we will switch to a binary classification problem: we will try to predict whether the setter will set in front of her or behind.

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

model = load("models/region_coordinate/tuned/xgb_hyperopt.joblib")
cm = confusion_matrix(y_test, model.predict(X_test))
_, ax = plt.subplots(figsize=(6, 6))
ax = sns.heatmap(cm, annot=True, cmap="Blues", fmt="d")
labels = ["L", "M", "R"]
ax.set_xticklabels(labels, fontsize=20)
ax.set_yticklabels(labels, fontsize=20)
ax.set_ylabel("Prediction", fontsize=20)
ax.set_xlabel("Ground Truth", fontsize=20)

# Target: binary_attacker
## Label encoding
This label presents a moderate unbalance.

In [ ]:
# 6. New approach: binarize the label
# 1: set in front
# 0: set back

from sklearn.model_selection import StratifiedShuffleSplit

df_binary = df_clean.copy()

target = "binarized_attacker"
df_binary.reset_index(inplace=True, drop=True)
y = df_binary[target]
X = df_binary.drop(columns=target)

sss = StratifiedShuffleSplit(n_splits=1, random_state=42, test_size=0.25)
train_index, test_index = next(sss.split(X, y))

X_train = X.iloc[train_index]
y_train = y.iloc[train_index]

X_test = X.iloc[test_index]
y_test = y.iloc[test_index]
y_train.value_counts(normalize=True)

## Preprocessing pipeline
This model uses the same preprocessing pipeline as the *region_coordinate* model.

## No-tuning results
Let's analyze again the performance of out-of-the-box algorithms. It seems that we would be able to predict the setter choice with 69% accuracy (+8% accuracy with respect to the 61% baseline).

In [ ]:
# Models definition
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    RandomForestClassifier,
    ExtraTreesClassifier,
    GradientBoostingClassifier,
    AdaBoostClassifier,
    HistGradientBoostingClassifier,
)
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from joblib import dump, load


DT = DecisionTreeClassifier(random_state=42)
RF = RandomForestClassifier(
    oob_score=True, random_state=42, warm_start=True, n_jobs=-1, bootstrap=True
)
ET = ExtraTreesClassifier(oob_score=True, bootstrap=True, random_state=42, n_jobs=-1)
lr = LogisticRegression(
    solver="saga", max_iter=2000, tol=1e-2, dual=False, random_state=42
)
kn = KNeighborsClassifier(n_jobs=-1)
lsvc = LinearSVC(random_state=42, tol=1e-2, max_iter=4000, dual=False)
svc = SVC(random_state=42, tol=1e-2, max_iter=4000, cache_size=1000)
gb = GradientBoostingClassifier(tol=1e-2, random_state=42)
ada = AdaBoostClassifier(random_state=42)
xgbm = xgb.XGBClassifier(use_label_encoder=False, eval_metric="logloss")
hgb = HistGradientBoostingClassifier(random_state=42, tol=1e-2)

classifiers = [DT, RF, ET, lr, kn, lsvc, svc, gb, ada, xgbm, hgb]
names = [
    "Decision Tree",
    "Random Forest",
    "Extra Trees",
    "Logistic Regression",
    "K-Neighbors",
    "Linear SVC",
    "SVC",
    "Gradient Boosting",
    "ADABoost",
    "XGBoost",
    "HistGradientBoosting",
]
save_name = ["dt", "rf", "et", "lr", "kn", "lsvc", "svc", "gb", "ada", "xgb", "hgb"]
# params = [DT_params, RF_params, ET_params, lr_params, kn_params, lsvc_params, svc_params, gb_params, ada_params, xgb_params]
parameters = [{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}]

In [ ]:
# WARNING: FIT TAKES LONG TIME!
folder = "models/binary_attacker/no_tuning/"

if TUNE:
    # WARNING: FIT TAKES LONG TIME!
    log_binary_df = helpclass.fit_models(
        X_train,
        y_train,
        X_test,
        y_test,
        preprocessor_full,
        classifiers,
        names,
        save_name,
        folder,
        parameters,
        cross_val_splits=5,
        verbose=False,
    )
else:
    log_binary_df = helpclass.eval_models(
        X_train,
        y_train,
        X_test,
        y_test,
        classifiers,
        names,
        save_name,
        folder,
        verbose=False,
    )

In [ ]:
display(
    log_binary_df[
        ["model", "accuracy", "precision", "recall", "f1", "accuracy_train"]
    ].sort_values(by="accuracy", ascending=False)
)

## No-tuning results (Reduced model)
Surprisingly, **the models score slightly better on the model using only a reduced set of features**.

In [ ]:
# WARNING: FIT TAKES LONG TIME!
folder = "models/binary_attacker/no_tuning_reduced/"

if TUNE:
    # WARNING: FIT TAKES LONG TIME!
    log_binary_reduced_df = helpclass.fit_models(
        X_train,
        y_train,
        X_test,
        y_test,
        preprocessor_full,
        classifiers,
        names,
        save_name,
        folder,
        parameters,
        cross_val_splits=5,
        verbose=False,
    )
else:
    log_binary_reduced_df = helpclass.eval_models(
        X_train,
        y_train,
        X_test,
        y_test,
        classifiers,
        names,
        save_name,
        folder,
        verbose=False,
    )

In [ ]:
display(
    log_binary_reduced_df[
        ["model", "accuracy", "precision", "recall", "f1", "accuracy_train"]
    ].sort_values(by="accuracy", ascending=False)
)

## XGBoost binary tuning on full set
We will use Hyperopt to explore the hyperparameter space of the XGBoost classifier. The tuned XGBoost algorithm achieves 0.711 accuracy and 0.673 f1-score (+5.2% and +5% with respect to the no-tuning case, respectively.)

In [ ]:
from hyperopt import hp

space = {
    "n_estimators": hp.quniform("n_estimators", 50, 500, 1),
    "max_depth": hp.quniform("max_depth", 4, 6, 1),
    "gamma": hp.uniform("gamma", 0, 3),
    "eta": hp.uniform("eta", 1e-4, 0.1),
    "reg_alpha": hp.uniform("reg_alpha", 0, 5),
    "reg_lambda": hp.uniform("reg_lambda", 1, 5),
    "min_child_weight": hp.uniform("min_child_weight", 1.7, 10),
    "seed": 42,
}

In [ ]:
X_tr_processed = preprocessor_full.fit_transform(X_train)
X_ts_preprocessed = preprocessor_full.transform(X_test)


def objective_xgb(explore_space):
    clf = xgb.XGBClassifier(
        use_label_encoder=False,
        subsample=0.7,
        eval_metric="mlogloss",
        n_estimators=int(explore_space["n_estimators"]),
        max_depth=int(explore_space["max_depth"]),
        gamma=explore_space["gamma"],
        eta=explore_space["eta"],
        reg_alpha=explore_space["reg_alpha"],
        min_child_weight=explore_space["min_child_weight"],
        reg_lambda=explore_space["reg_lambda"],
    )

    evaluation = [(X_tr_processed, y_train), (X_ts_preprocessed, y_test)]

    clf.fit(
        X_tr_processed,
        y_train,
        eval_set=evaluation,
        eval_metric="logloss",
        early_stopping_rounds=30,
        verbose=False,
    )
    scores = cross_val_score(clf, X_tr_processed, y_train, scoring="accuracy", cv=5)
    scores_avg = scores.mean()

    # print (f"5-CV avg SCORE: {scores_avg}")
    return {"loss": -scores_avg, "status": STATUS_OK}

In [ ]:
if TUNE:
    trials = Trials()
    best_hyperparams = fmin(
        fn=objective_xgb, space=space, algo=tpe.suggest, max_evals=150, trials=trials
    )
    print("The best hyperparameters are : ", "\n")
    print(best_hyperparams)

In [ ]:
import xgboost as xgb

folder = "models/binary_attacker/tuned/"
model = "xgb_hyperopt"
name = "XGBoost"

if TUNE:
    param_grid = {
        "classifier__subsample": [0.7],
        "classifier__n_estimators": [225],
        "classifier__max_depth": [6],
        "classifier__min_child_weight": [3.296],
        "classifier__gamma": [1.4],
        "classifier__eta": [0.001],
        "classifier__lambda": [2.144],
        "classifier__alpha": [1.495],
        #     'classifier__subsample': [0.7],
        #     'classifier__n_estimators': [300],
        #     'classifier__max_depth': [4],
        #     'classifier__min_child_weight': [3.084],
        #     'classifier__gamma': [0.657],
        #     'classifier__eta': [0.0004],
        #     'classifier__lambda': [2.229],
        #     'classifier__alpha': [2.720],
    }

    complete_pipe = Pipeline(
        steps=[
            ("preprocessor", preprocessor_full),
            (
                "classifier",
                xgb.XGBClassifier(use_label_encoder=False, eval_metric="logloss"),
            ),
        ]
    )
    CV = GridSearchCV(complete_pipe, param_grid, n_jobs=-1, cv=5)
    CV.fit(X_train, y_train)

    print(CV.best_params_)
    print(CV.best_score_)
    dump(CV.best_estimator_, f"{folder}{model}.joblib")
    report_model_performance(
        X_train,
        X_test,
        y_train,
        y_test,
        CV.best_estimator_,
        name,
        param_grid=param_grid,
        verbose=True,
    )
else:
    loaded_model = load(f"{folder}{model}.joblib")
    print(loaded_model.named_steps["classifier"])
    print("\n")
    report_model_performance(
        X_train, X_test, y_train, y_test, loaded_model, name, verbose=True
    )

### XGBoost binary tuning on reduced features set
On the reduced feature set, the tuned XGBoost records 0.709 accuracy and 0.672 f1-score (+2.9% and +2.2% with respect to the untuned model, respectively).

In [ ]:
X_tr_processed = preprocessor_reduced.fit_transform(X_train)
X_ts_preprocessed = preprocessor_reduced.transform(X_test)


def objective_xgb(explore_space):
    clf = xgb.XGBClassifier(
        random_state=42,
        use_label_encoder=False,
        subsample=0.7,
        eval_metric="logloss",
        n_estimators=int(explore_space["n_estimators"]),
        max_depth=int(explore_space["max_depth"]),
        gamma=explore_space["gamma"],
        eta=explore_space["eta"],
        reg_alpha=explore_space["reg_alpha"],
        min_child_weight=explore_space["min_child_weight"],
        reg_lambda=explore_space["reg_lambda"],
    )

    evaluation = [(X_tr_processed, y_train), (X_ts_preprocessed, y_test)]

    clf.fit(
        X_tr_processed,
        y_train,
        eval_set=evaluation,
        eval_metric="logloss",
        early_stopping_rounds=30,
        verbose=False,
    )
    scores = cross_val_score(clf, X_tr_processed, y_train, scoring="accuracy", cv=5)
    accuracy_avg = scores.mean()

    print(f"5-CV Accuracy SCORE: {accuracy_avg}")
    return {"loss": -accuracy_avg, "status": STATUS_OK}

In [ ]:
if TUNE:
    trials = Trials()
    best_hyperparams = fmin(
        fn=objective_xgb, space=space, algo=tpe.suggest, max_evals=150, trials=trials
    )
    print("The best hyperparameters are : ", "\n")
    print(best_hyperparams)

In [ ]:
import xgboost as xgb

folder = "models/binary_attacker/tuned/"
model = "xgb_hyperopt_reduced"
name = "XGBoost"

if TUNE:
    param_grid = {
        "classifier__subsample": [0.7],
        "classifier__n_estimators": [131],
        "classifier__max_depth": [4],
        "classifier__min_child_weight": [3.7],
        "classifier__gamma": [0.008],
        "classifier__eta": [0.0012],
        "classifier__lambda": [4.36],
        "classifier__alpha": [0.11],
    }

    complete_pipe = Pipeline(
        steps=[
            ("preprocessor", preprocessor_reduced),
            (
                "classifier",
                xgb.XGBClassifier(use_label_encoder=False, eval_metric="logloss"),
            ),
        ]
    )
    CV = GridSearchCV(complete_pipe, param_grid, n_jobs=-1, cv=5)
    CV.fit(X_train, y_train)

    print(CV.best_params_)
    print(CV.best_score_)
    dump(CV.best_estimator_, f"{folder}{model}.joblib")
    report_model_performance(
        X_train,
        X_test,
        y_train,
        y_test,
        CV.best_estimator_,
        name,
        param_grid=param_grid,
        verbose=True,
    )
else:
    loaded_model = load(f"{folder}{model}.joblib")
    print(loaded_model.named_steps["classifier"])
    print("\n")
    report_model_performance(
        X_train, X_test, y_train, y_test, loaded_model, name, verbose=True
    )

## SVC tuning on full set
Let's try tuning other algorithm to achieve better performance, and possibly stacking multiple classifiers.

Exploring the hyperspace to maximize the f1-score, we obtained an accuracy of 0.654 (+2.7%) and an f1-score of (+6.5%). The **performance on the test set is still lower than for the XGBoost classifier**.


In [ ]:
from hyperopt import hp

space = {
    "C": hp.loguniform("C", -2, 1),
    "gamma": hp.loguniform("gamma", -2, 1),
}

In [ ]:


X_tr_processed = preprocessor_full.fit_transform(X_train)
X_ts_preprocessed = preprocessor_full.transform(X_test)


def objective_svc(explore_space):
    clf = SVC(
        random_state=42,
        cache_size=1000,
        tol=1e-3,
        kernel="rbf",
        probability=True,
        C=float(explore_space["C"]),
        gamma=float(explore_space["gamma"]),
    )

    clf.fit(X_tr_processed, y_train)
    scores = cross_val_score(clf, X_tr_processed, y_train, scoring="f1", cv=5)
    accuracy_avg = scores.mean()

    print(f"5-CV Accuracy SCORE: {accuracy_avg:.4f}")
    return {"loss": -accuracy_avg, "status": STATUS_OK}

In [ ]:
if TUNE:
    trials = Trials()
    best_hyperparams = fmin(
        fn=objective_svc, space=space, algo=tpe.suggest, max_evals=200, trials=trials
    )
    print("The best hyperparameters are : ", "\n")
    print(best_hyperparams)
    print(f"Loss max: {min(trials.losses())}")

In [ ]:
folder = "models/binary_attacker/tuned/"
model = "svc_hyperopt"
name = "SVC"

if TUNE:
    param_grid = {
        "classifier__C": [2.6675],
        "classifier__class_weight": [None],
        "classifier__gamma": [0.146],
        "classifier__kernel": ["rbf"],
    }

    complete_pipe = Pipeline(
        steps=[
            ("preprocessor", preprocessor_full),
            (
                "classifier",
                SVC(random_state=42, cache_size=1000, tol=1e-2, probability=True),
            ),
        ]
    )
    CV = GridSearchCV(complete_pipe, param_grid, n_jobs=-1, cv=5)
    CV.fit(X_train, y_train)

    print(CV.best_params_)
    print(CV.best_score_)
    dump(CV.best_estimator_, f"{folder}{model}.joblib")
    report_model_performance(
        X_train,
        X_test,
        y_train,
        y_test,
        CV.best_estimator_,
        name,
        param_grid=param_grid,
        verbose=True,
    )
else:
    loaded_model = load(f"{folder}{model}.joblib")
    print(loaded_model.named_steps["classifier"])
    print("\n")
    report_model_performance(
        X_train, X_test, y_train, y_test, loaded_model, name, verbose=True
    )

### SVC on reduced set
For the reduced set, the optimization of the f1-score led to a +0.2% in accuracy and +6.8% in f1-score.

In [ ]:
X_tr_processed = preprocessor_reduced.fit_transform(X_train)
X_ts_preprocessed = preprocessor_reduced.transform(X_test)


def objective_svc(explore_space):
    clf = SVC(
        random_state=42,
        cache_size=1000,
        tol=5e-3,
        kernel="rbf",
        C=float(explore_space["C"]),
        gamma=float(explore_space["gamma"]),
        probability=True,
    )

    clf.fit(X_tr_processed, y_train)
    scores = cross_val_score(clf, X_tr_processed, y_train, scoring="f1", cv=5)
    accuracy_avg = scores.mean()

    # print (f"5-CV Accuracy SCORE: {accuracy_avg:.4f}")
    return {"loss": -accuracy_avg, "status": STATUS_OK}

In [ ]:
if TUNE:
    trials = Trials()
    best_hyperparams = fmin(
        fn=objective_svc, space=space, algo=tpe.suggest, max_evals=300, trials=trials
    )
    print("The best hyperparameters are : ", "\n")
    print(best_hyperparams)
    print(f"Loss max: {min(trials.losses())}")

In [ ]:
folder = "models/binary_attacker/tuned/"
model = "svc_hyperopt_reduced"
name = "SVC"

if TUNE:
    param_grid = {
        "classifier__C": [2.70],
        "classifier__class_weight": [None],
        "classifier__gamma": [0.343],
        "classifier__kernel": ["rbf"],
    }

    complete_pipe = Pipeline(
        steps=[
            ("preprocessor", preprocessor_reduced),
            (
                "classifier",
                SVC(random_state=42, cache_size=1000, tol=5e-2, probability=True),
            ),
        ]
    )
    CV = GridSearchCV(complete_pipe, param_grid, n_jobs=-1, cv=5)
    CV.fit(X_train, y_train)

    print(CV.best_params_)
    print(CV.best_score_)
    dump(CV.best_estimator_, f"{folder}{model}.joblib")
    helpclass.report_model_performance(
        X_train,
        X_test,
        y_train,
        y_test,
        CV.best_estimator_,
        name,
        param_grid=param_grid,
        verbose=True,
    )
else:
    loaded_model = load(f"{folder}{model}.joblib")
    print(loaded_model.named_steps["classifier"])
    print("\n")
    report_model_performance(
        X_train, X_test, y_train, y_test, loaded_model, name, verbose=True
    )

## Logistic regression tuning on full set
In this case, tuning did not lead to any improvement (about -1.5% for both accuracy and f1-score).

In [ ]:
from hyperopt import hp


space = {
    "C": hp.loguniform("C", -2, 0),
}

X_tr_processed = preprocessor_full.fit_transform(X_train)
X_ts_preprocessed = preprocessor_full.transform(X_test)


def objective_logit(explore_space):
    clf = LogisticRegression(
        random_state=42,
        max_iter=4000,
        tol=1e-3,
        dual=False,
        solver="liblinear",
        penalty="l2",
        C=float(explore_space["C"]),
    )

    clf.fit(X_tr_processed, y_train)
    scores = cross_val_score(clf, X_tr_processed, y_train, scoring="f1", cv=5)
    accuracy_avg = scores.mean()

    # print (f"5-CV Accuracy SCORE: {accuracy_avg:.4f}")
    return {"loss": -accuracy_avg, "status": STATUS_OK}

In [ ]:
if TUNE:
    trials = Trials()
    best_hyperparams = fmin(
        fn=objective_logit, space=space, algo=tpe.suggest, max_evals=300, trials=trials
    )
    print("The best hyperparameters are : ", "\n")
    print(best_hyperparams)
    print(f"Loss max: {min(trials.losses())}")

In [ ]:
folder = "models/binary_attacker/tuned/"
model = "lr_hyperopt"
name = "Logistic Regression"

if TUNE:
    param_grid = {
        "classifier__C": [0.679],
    }

    complete_pipe = Pipeline(
        steps=[
            ("preprocessor", preprocessor_full),
            (
                "classifier",
                LogisticRegression(
                    random_state=42,
                    max_iter=2000,
                    tol=1e-3,
                    dual=False,
                    solver="liblinear",
                    penalty="l2",
                ),
            ),
        ]
    )
    CV = GridSearchCV(complete_pipe, param_grid, n_jobs=-1, cv=5)
    CV.fit(X_train, y_train)

    print(CV.best_params_)
    print(CV.best_score_)
    dump(CV.best_estimator_, f"{folder}{model}.joblib")
    helpclass.report_model_performance(
        X_train,
        X_test,
        y_train,
        y_test,
        CV.best_estimator_,
        name,
        param_grid=param_grid,
        verbose=True,
    )
else:
    loaded_model = load(f"{folder}{model}.joblib")
    print(loaded_model.named_steps["classifier"])
    print("\n")
    report_model_performance(
        X_train, X_test, y_train, y_test, loaded_model, name, verbose=True
    )

### Logistic regression on reduced set

In [ ]:
from hyperopt import hp


space = {
    "C": hp.loguniform("C", -3, 0),
    "l1_ratio": hp.uniform("l1_ratio", 0, 1),
}

X_tr_processed = preprocessor_reduced.fit_transform(X_train)
X_ts_preprocessed = preprocessor_reduced.transform(X_test)


def objective_logit(explore_space):
    clf = LogisticRegression(
        random_state=42,
        max_iter=2000,
        tol=1e-3,
        dual=False,
        solver="saga",
        penalty="elasticnet",
        n_jobs=-1,
        C=float(explore_space["C"]),
        l1_ratio=float(explore_space["l1_ratio"]),
    )

    clf.fit(X_tr_processed, y_train)
    scores = cross_val_score(clf, X_tr_processed, y_train, scoring="f1", cv=5)
    accuracy_avg = scores.mean()

    # print (f"5-CV Accuracy SCORE: {accuracy_avg:.4f}")
    return {"loss": -accuracy_avg, "status": STATUS_OK}

In [ ]:
if TUNE:
    trials = Trials()
    best_hyperparams = fmin(
        fn=objective_logit, space=space, algo=tpe.suggest, max_evals=400, trials=trials
    )
    print("The best hyperparameters are : ", "\n")
    print(best_hyperparams)
    print(f"Loss max: {min(trials.losses())}")

In [ ]:
folder = "models/binary_attacker/tuned/"
model = "lr_hyperopt_reduced"
name = "Logistic Regression"

if TUNE:
    param_grid = {
        "classifier__C": [0.706],
        "classifier__l1_ratio": [0.16],
    }

    complete_pipe = Pipeline(
        steps=[
            ("preprocessor", preprocessor_full),
            (
                "classifier",
                LogisticRegression(
                    random_state=42,
                    max_iter=2000,
                    tol=5e-3,
                    dual=False,
                    solver="saga",
                    penalty="elasticnet",
                    n_jobs=-1,
                ),
            ),
        ]
    )
    CV = GridSearchCV(complete_pipe, param_grid, n_jobs=-1, cv=5)
    CV.fit(X_train, y_train)

    print(CV.best_params_)
    print(CV.best_score_)
    dump(CV.best_estimator_, f"{folder}{model}.joblib")
    helpclass.report_model_performance(
        X_train,
        X_test,
        y_train,
        y_test,
        CV.best_estimator_,
        name,
        param_grid=param_grid,
        verbose=True,
    )
else:
    loaded_model = load(f"{folder}{model}.joblib")
    print(loaded_model.named_steps["classifier"])
    print("\n")
    report_model_performance(
        X_train, X_test, y_train, y_test, loaded_model, name, verbose=True
    )

## K-Neighbors tuning on full set
Tuning leads to a +4.2% accuracy and +4% f1-score. However, K-Neighbors (as Logistic Regression) in this case led only to marginal improvements with respect to the baseline.

In [ ]:
from hyperopt import hp

space = {
    "n_neighbors": hp.uniform("n_neighbors", 5, 40),
    "weights": hp.choice("weights", ["uniform", "distance"]),
    "leaf_size": hp.uniform("leaf_size", 10, 40),
}


X_tr_processed = preprocessor_full.fit_transform(X_train)
X_ts_preprocessed = preprocessor_full.transform(X_test)


def objective_knn(explore_space):
    clf = KNeighborsClassifier(
        n_jobs=-1,
        n_neighbors=int(explore_space["n_neighbors"]),
        weights=explore_space["weights"],
        leaf_size=int(explore_space["leaf_size"]),
    )

    clf.fit(X_tr_processed, y_train)
    scores = cross_val_score(clf, X_tr_processed, y_train, scoring="f1", cv=5)
    accuracy_avg = scores.mean()

    # print (f"5-CV Accuracy SCORE: {accuracy_avg:.4f}")
    return {"loss": -accuracy_avg, "status": STATUS_OK}

In [ ]:
if TUNE:
    trials = Trials()
    best_hyperparams = fmin(
        fn=objective_knn, space=space, algo=tpe.suggest, max_evals=300, trials=trials
    )
    print("The best hyperparameters are : ", "\n")
    print(best_hyperparams)
    print(f"Loss max: {min(trials.losses())}")

In [ ]:
folder = "models/binary_attacker/tuned/"
model = "knn_hyperopt"
name = "KNN"

if TUNE:
    param_grid = {
        "classifier__leaf_size": [34],
        "classifier__n_neighbors": [7],
        "classifier__weights": ["distance"],
    }

    complete_pipe = Pipeline(
        steps=[
            ("preprocessor", preprocessor_full),
            ("classifier", KNeighborsClassifier(n_jobs=-1)),
        ]
    )
    CV = GridSearchCV(complete_pipe, param_grid, n_jobs=-1, cv=5)
    CV.fit(X_train, y_train)

    print(CV.best_params_)
    print(CV.best_score_)
    dump(CV.best_estimator_, f"{folder}{model}.joblib")
    report_model_performance(
        X_train,
        X_test,
        y_train,
        y_test,
        CV.best_estimator_,
        name,
        param_grid=param_grid,
        verbose=True,
    )
else:
    loaded_model = load(f"{folder}{model}.joblib")
    print(loaded_model.named_steps["classifier"])
    print("\n")
    report_model_performance(
        X_train, X_test, y_train, y_test, loaded_model, name, verbose=True
    )

### K-Neighbors on reduced set
After tuning: +7% accuracy and +6.7% f1-score (better than the full model!)

In [ ]:
from hyperopt import hp
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from hyperopt import STATUS_OK, Trials, fmin, tpe

space = {
    "n_neighbors": hp.uniform("n_neighbors", 5, 40),
    "weights": hp.choice("weights", ["uniform", "distance"]),
    "leaf_size": hp.uniform("leaf_size", 10, 40),
}


X_tr_processed = preprocessor_reduced.fit_transform(X_train)
X_ts_preprocessed = preprocessor_reduced.transform(X_test)


def objective_knn(explore_space):
    clf = KNeighborsClassifier(
        n_jobs=-1,
        n_neighbors=int(explore_space["n_neighbors"]),
        weights=explore_space["weights"],
        leaf_size=int(explore_space["leaf_size"]),
    )

    clf.fit(X_tr_processed, y_train)
    scores = cross_val_score(clf, X_tr_processed, y_train, scoring="f1", cv=5)
    accuracy_avg = scores.mean()

    # print (f"5-CV Accuracy SCORE: {accuracy_avg:.4f}")
    return {"loss": -accuracy_avg, "status": STATUS_OK}

In [ ]:
if TUNE:
    trials = Trials()
    best_hyperparams = fmin(
        fn=objective_knn, space=space, algo=tpe.suggest, max_evals=300, trials=trials
    )
    print("The best hyperparameters are : ", "\n")
    print(best_hyperparams)
    print(f"Loss max: {min(trials.losses())}")

In [ ]:
folder = "models/binary_attacker/tuned/"
model = "knn_hyperopt_reduced"
name = "KNN"

if TUNE:
    param_grid = {
        "classifier__leaf_size": [15],
        "classifier__n_neighbors": [25],
        "classifier__weights": ["distance"],
    }

    complete_pipe = Pipeline(
        steps=[
            ("preprocessor", preprocessor_reduced),
            ("classifier", KNeighborsClassifier(n_jobs=-1)),
        ]
    )
    CV = GridSearchCV(complete_pipe, param_grid, n_jobs=-1, cv=5)
    CV.fit(X_train, y_train)

    print(CV.best_params_)
    print(CV.best_score_)
    dump(CV.best_estimator_, f"{folder}{model}.joblib")
    helpclass.report_model_performance(
        X_train,
        X_test,
        y_train,
        y_test,
        CV.best_estimator_,
        name,
        param_grid=param_grid,
        verbose=True,
    )
else:
    loaded_model = load(f"{folder}{model}.joblib")
    print(loaded_model.named_steps["classifier"])
    print("\n")
    report_model_performance(
        X_train, X_test, y_train, y_test, loaded_model, name, verbose=True
    )

## Ensemble results
In this case, **performance with an ensemble (XGBoost + KNN + SVC, on reduced set, with hard voting) is lower than the performance of the single XGBoost Classifier**.

<code>
estimators = [
    ("xgb", xgbc.named_steps["classifier"]),
    ("knn", knn.named_steps["classifier"]),
    ("svc", svc.named_steps["classifier"])
]

pipe = Pipeline([
    ("preprocessing", preprocessor_reduced),
    ("classifier", VotingClassifier(estimators, voting="hard"))
])
</code>


In [ ]:
from sklearn.ensemble import VotingClassifier

svc = load("models/binary_attacker/tuned/svc_hyperopt_reduced.joblib")
knn = load("models/binary_attacker/tuned/knn_hyperopt_reduced.joblib")
xgbc = load("models/binary_attacker/tuned/xgb_hyperopt_reduced.joblib")
name = "ensamble"

estimators = [
    ("xgb", xgbc.named_steps["classifier"]),
    ("knn", knn.named_steps["classifier"]),
    ("svc", svc.named_steps["classifier"]),
]

pipe = Pipeline(
    [
        ("preprocessing", preprocessor_reduced),
        ("classifier", VotingClassifier(estimators, voting="hard")),
    ]
)

mod = pipe.fit(X_train, y_train)
helpclass.report_model_performance(
    X_train, X_test, y_train, y_test, mod, name, param_grid={}, verbose=True
)

## Conclusions on *binary_attacker* target
Once again the results are positive, but still not quite accurate. The classifier (XGBoost) presents a lot of "false-negatives" (predicts Back-0 while the ground truth is Front-1).

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

model = load("models/binary_attacker/tuned/xgb_hyperopt.joblib")
cm = confusion_matrix(y_test, model.predict(X_test))
_, ax = plt.subplots(figsize=(6, 6))
ax = sns.heatmap(cm, annot=True, cmap="Blues", fmt="d")
labels = ["Back", "Front"]
ax.set_xticklabels(labels, fontsize=20)
ax.set_yticklabels(labels, fontsize=20)
ax.set_ylabel("Prediction", fontsize=20)
ax.set_xlabel("Ground Truth", fontsize=20)

# Model interpretation
The best models I found for this dataset are tree-based (like XGBoost and Random Forest). With complex models, it is not always clear which features are contributing to a prediction. For that, I will use the [SHAP](https://github.com/slundberg/shap) (SHapley Additive exPlanations) package, based on the Shapley concept, which show the impact of each feature into the model prediction. It's important to highlight that this analysis is referred only to the model prediction, and not the ground truth.

I will use the XGBoost reduced model developed for the *region_coordinate* target.

In [ ]:
import shap

model = load("models/region_coordinate/tuned/xgb_hyperopt.joblib")

features = get_features_from_transformer_list(
    model.named_steps["preprocessor"].named_steps["transformer"]
)

# load JS visualization code to notebook
shap.initjs()


explainer = shap.TreeExplainer(model["classifier"])
test_X = pd.DataFrame(data=model["preprocessor"].transform(X_test), columns=features)
shap_values = explainer.shap_values(test_X)
categories = pd.Series(enc_target.classes_)
print(f"Targets:\n{categories}")

## Global interpretability
### Left (L)

Let's take a look at the importance of different features when the setter set LEFT (L, encoded as class 0).

- ***set_x* seems to have the largest impact on the prediction**. With large values of *set_x* (in pink in the plot, correspond to the setter moving to the R side of the court), the model tend to classify the L output class as less likely. This means that **the model learned that as the setter moves to the right side of the court, she tends to NOT set the ball to the left side**. **The model thinks she is a setter that plays the short distance**.
- *F_Block_height*: the taller the block of the front player (setter or opposite), the less the model thinks she will set to the L-side attacker.
- *B_Block_height*: the taller the block of the back player (outside hitters), the more the model thinks she will set to the L-side attacker.
- *call*: for calls behind the setter (i.e., K2 or KF, large values of *call* feature) the model thinks the play with the L-player is less likely, while it is more likely for calls K7 (low value of *call*).
- *rotation_home*: large values (i.e., three-attackers rotations) the L-attacker is less likely than in two-attackers rotations.

In [ ]:
shap.summary_plot(
    shap_values[0],
    test_X,
)

### Right (R)

For the R-attacker (encoded as class 2):
- *rotation_home*: the feature with most impact. Very high value of *rotation_home* (rotation 1 according to the encoding) indicate much less probability of serving the R side, while intermediate values (rotation 6, 5) greatly increase its probability.
- *call*: Large values of call (Kf and K2 especially) are associated with very large probability of attacking the R side of the net.
- *set_y*: increasing *set_y* (setter location closer and closer to the net) is associated with lower R-side probability and vice-versa.

In [ ]:
shap.summary_plot(shap_values[2], test_X)

### Middle (M)

For the M-side attacks (encoded as class 1):
- *rotation_home*: the feature with most impact. Very high value of *rotation_home* (rotation 1 mostly, according to the encoding, and mildly for rotation 6) indicate much higher probability of serving the M-attacker. Rotation 5 is associated with less probability.
- *B_Block_height*: **Small values of *B_Block_height* are associated with large probability of serving the M-attacker**.
- *set_y*: increasing *set_y* (setter location closer and closer to the net) is associated with higher M-attacker probability.

In [ ]:
shap.summary_plot(shap_values[1], test_X)

## Partial dependence plots
### Importance of *set_x* in L/R decision

We can look at the partial dependence plot, to show the relationship of one or two variables on the predicted outcome.

For example, looking at the effect of *set_x* on the **L-side of the net** Shapley values, as the setter moves to the R side (increasing *set_x* on the x-axis) the lower its Shapley value. The model thinks that when the setter moves to the R-side, she will not set to the L-side of the net (**setter plays the short distance**).

In [ ]:
shap.dependence_plot("set_x", shap_values[0], X_test[features])

Conversely, when looking at the **R-side** of the net, as the setter moves right, a play with the R-side is more likely. **As she moves left and past the left-half of the court, she will not play with the R-side, and this is especially true if the F-blocker is short** (blue dots).

In [ ]:
shap.dependence_plot("set_x", shap_values[2], X_test[features])

### Importance of *set_y* in M decisions

For the central area of the net (M), attacking the M-area becomes increasingly more likely as the setter moves closer to the net (larger *set_y*).

In [ ]:
shap.dependence_plot("set_y", shap_values[1], X_test[features])

For the C-area of the court, with call K7 it is unlikely for the C-area to be served, especially if *reception_x* is small (reception on the R-side of the court, zone 1).

In [ ]:
shap.dependence_plot("call", shap_values[1], X_test[features])

## Local predictability: single side-out prediction
It is also possible to **explain the model prediction for a single side-out event**. In red, the factors that increase the estimation *f(x)* with respect to the *base_value*, in blue the factors that decrease the estimated probability. For example:

In [ ]:
example_index = 25
display(X_test[features].iloc[[example_index]])
shap.initjs()

R-Side estimation:

In [ ]:
shap.force_plot(
    explainer.expected_value[0],
    shap_values[0][example_index, :],
    X_test[features].iloc[example_index, :],
    link="logit",
)

M-Side estimation:

In [ ]:
shap.force_plot(
    explainer.expected_value[1],
    shap_values[1][example_index, :],
    X_test[features].iloc[example_index, :],
    link="logit",
)

L-Side estimation:

In [ ]:
shap.force_plot(
    explainer.expected_value[2],
    shap_values[2][example_index, :],
    X_test[features].iloc[example_index, :],
    link="logit",
)

Which side of the net got attacked in the end?

In [ ]:
print(
    f"Side attacked: {pd.Series(enc_target.inverse_transform(y_test)).iloc[example_index]}"
)

In this case I was "lucky" with the prediction!

# General conclusions
The model delivered **reasonable performance above the baseline, but not at a satisfactory levels** (80% accuracy on the *region_combination* model would be considered excellent and dependable). The issues can be summarized as:
- **Not enough data-points** to develop the models
- Possibly missing some important features that can better help to classify the instances
- The **process (setter choosing which area to attack) might be at least partially aleatory** in nature
- The **idea of simple decision trees for players does not seem ready to work** (yet?)
- **Gradient boosting models (like XGBoost) delivered the best performance, followed by RandomForest**

Some general conclusion on the setter behavior could be extracted nevertheless:
- importance of location of the court (*set_x*) in attacking L/R side (**setter plays the short distance**)
- **the setter seems to prefer to attack the short block**. This is often one of the most important factor in her decision
- **very different choices in rotation 1** (prefer to attack the center of the court)
- **importance of the distance from the net (*set_y*) in inhibiting the attack of the center of the court (M)**



# Suggestions and next steps
To solve the problems above, a few idea come to mind:
- Use a different training/test split, increasing the dimension of the test split to reduce the variance of the estimated performance. This must be paired with the need of bootstrapping during the construction of the model
- **Use other datasets similar in nature for the training of the models** (this could be much more useful with a **neural network**, allowing the re-use of some trained layers)
- The use of other boosting models (LightGBM, Catboost) does not seem to be the answer to this problems
- **Introduce new features** that summarize the "history" that brought to the current instance. For example:
    1. Players attack past performance in the current set or match
    2. Players number of attacks in the current set or match
    3. Previous set choice in the last side-out rally
    4. Opponent setter set choice in the last side-out rally (to identify "mirror" setters)
